# SelectQAOA: Regression Test Case Selection Using QAOA
Regression testing is an important part of the software development process in software engineering. It is a practice aimed at identifying any regression, which are the emergence of new defects or issues in a software application following changes, enhancements, or updates made to the source code. In other words, regression testing focuses on how changes made to the software can affect the correct behavior of existing features. Regression testing is particularly important in agile software development environments, where changes are made frequently and rapidly. This practice helps ensure that the software remains stable and reliable as it evolves over time. Ideal regression testing would re-run all the available test cases of a given software system. However, in addition to being potentially very costly, this could even be impractical in some case. In this scenario, test case selection is one of the most widely investigated test suite optimization approaches.
Test case selection focuses on selecting a subset from an initial test suite to test software changes, i.e., to test whether unmodified parts of a program continue to work correctly after changes involving other parts. Various techniques, such as Integer Programming, symbolic execution, data flow analysis, dependence graph-based methods, and flow graph-based approaches, can be employed to identify the modified portions of the software. Once test cases covering the unchanged program segments are pinpointed using a specific technique, an optimization algorithm (e.g., additional greedy, DIV-GA,
SelectQA, BootQA or SelectQAOA) can select a minimal set of these test cases based on certain testing criteria (e.g., branch coverage). The ultimate aim is to reduce the expenses associated with regression testing.

In [ ]:
#this cell contains all the imports needed by the pipeline
#to run it on the browser: jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
import os
import json
import time

import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import statistics
import csv

from scipy.stats import mannwhitneyu, shapiro
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import BackendSampler
from qiskit_algorithms import QAOA
from qiskit_algorithms.utils import algorithm_globals
from qiskit_aer import Aer, AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_ibm_runtime.fake_provider import FakeVigoV2
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, fcluster
from collections import defaultdict
from scipy.spatial.distance import cdist

## The pipeline dataset
To experiment the performance of the proposed solution by this work and to compare its results to those of state-of-the-art solutions, 4 public programs have been downloaded from the SIR website. SIR is a repository of software-related artifacts meant to support rigorous controlled experimentation with program analysis and software testing techniques, and education in controlled experimentation. 

### Chosen SIR Programs
The programs that will be used for experimentation have all been written in C and are:
- flex (a program that generates a lexical analysis program, based on regular expressions and C statement contained in one or more input files);
- grep (a useful program to search form matching patterns in a file);
- gzip (a program that substitute a file, generally text files or web pages, with their compressed version)
- sed (a powerful program for stream text editing).

### Needed information
The information needed by the quantum algorithm to work on every one of the 4 programs are:
- a fault matrix: it indicates whether a precise test case already found, during previous execution, a bug in the source code or not;
- execution cost: it indicates the execution cost of any test case of the suite;
- statement coverage: it indicates statement coverage information for every test case. 

All this information have been gathered through previous experimentation on the four programs mentioned above and written in files organized in the SIR_Programs folder. So, the first goal of the project will be gathering data from these files for computational purposes.

In [ ]:
#this cell contains all variable definitions that will be useful throughout the entire project
sir_programs = ["flex","grep","gzip","sed"]
sir_programs_tests_number = {"flex":567,"grep":806,"gzip":214,"sed":360}
sir_programs_end_lines = {"flex":14192,"grep":13281,"gzip":6701,"sed":7118}
alpha = 0.5
experiments = 30

In [ ]:
def json_keys_to_int(d):
    if isinstance(d, dict):
        return {int(k) if k.isdigit() else k: json_keys_to_int(v) for k, v in d.items()}
    elif isinstance(d, list):
        return [json_keys_to_int(i) for i in d]
    else:
        return d


with open("datasets/sir_programs/executed_lines_test_by_test.json", "r") as file:
    #dictionary that, for each sir program, associates at each LINE of that program the LIST of TESTS COVERING it
    executed_lines_test_by_test = json_keys_to_int(json.load(file)) #{program1:{line:[tci,tcj,...,tck],line2:...}
with open("datasets/sir_programs/faults_dictionary.json", "r") as file:
    #dictionary that associates at each SIR PROGRAM the LIST of PAST FAULT COVERAGE VALUES ORDERED BY TEST 
    faults_dictionary = json.load(file) #{program1:[fault_cov_tc1,fault_cov_tc2,...,fault_cov_tcn],program2:...}
with open("datasets/sir_programs/test_coverage_line_by_line.json", "r") as file:
    #dictionary that, for each sir program, associates at each TEST of that program the LIST of LINES COVERED by it
    test_coverage_line_by_line = json_keys_to_int(json.load(file)) #{program1:{tc1:[linei,linej,...,linek],tc2:...}
with open("datasets/sir_programs/test_cases_costs.json", "r") as file:
    #dictionary that, for each sir program, associates at each TEST its EXECUTION COST
    test_cases_costs = json_keys_to_int(json.load(file)) #{program1:{tc1:ex_cost1,tc2:ex_cost2,...,tcn:ex_costn},program2:...}
with open("datasets/sir_programs/total_program_lines.json", "r") as file:
    #dictionary wich associates at each SIR PROGRAM its size in terms of NUMBER OF ITS LINES
    total_program_lines = json.load(file) #{program1:tot_lines_program1,program2:tot_lines_program2,program3:...}

## Quantum Approximate Optimization Algorithm (QAOA)
The Quantum Approximate Optimization Algorithm (QAOA) is a quantum algorithm designed to tackle combinatorial optimization problems by combining classical and quantum computing techniques. It encodes the optimization problem into a Quantum Unconstrained Binary Optimization (QUBO) format and constructs a parameterized quantum circuit that alternates between applying the problem Hamiltonian, which represents the optimization goal, and a mixing Hamiltonian, which promotes exploration of the solution space. The parameters of this circuit are optimized using classical optimization methods to maximize the likelihood of measuring the optimal solution. Once the parameters are tuned, the circuit is executed on a quantum device or simulator to obtain candidate solutions, making QAOA particularly suitable for NP-hard problems like graph partitioning and maximum cut. This hybrid approach highlights the potential of quantum computing to provide advantages in solving complex optimization challenges.

## QUBO Problems
A Quadratic Unconstrained Binary Optimization (QUBO) problem is a type of mathematical problem where we seek to find the best combination of binary values (0 or 1) for a set of variables to minimize or maximize an objective function. In other words, we are looking for the optimal solution among all possible variable combinations that satisfies certain constraints and makes the objective function as small as possible.

Weights for constraints (or penalty coefficients) are used in QUBO problems to assign a numerical value to the constraints and influence the optimization process. These weights are important because they allow for managing the priority and relative importance of constraints within the optimization problem.

In [ ]:
def num_of_covered_lines(sir_program,test_cases):
    covered_lines = set()
    
    for test_case in test_cases:
        try:
            for covered_line in test_coverage_line_by_line[sir_program][test_case]:
                covered_lines.add(covered_line)
        except:
            continue
    
    return len(covered_lines)

clusters_dictionary = dict()

for sir_program in sir_programs:
    tot_test_cases = sir_programs_tests_number[sir_program]
    
    # from {..., test_case_i : [cov_stmts], ...} to [..., #_stmt_cov_i, ...]
    test_cases_stmt_cov = []
    for test_case in test_coverage_line_by_line[sir_program].keys():
        test_cases_stmt_cov.append(len(test_coverage_line_by_line[sir_program][test_case]))
    suite_stmt_cov = sum(test_cases_stmt_cov)
    
    # Normalize data
    data = np.column_stack((list(test_cases_costs[sir_program].values()),faults_dictionary[sir_program],test_cases_stmt_cov))
    scaler = StandardScaler()
    normalized_data = scaler.fit_transform(data)

    num_clusters = 0
    
    #ATTENTION: this number also depends on the QAOA simulator/machine you will use
    if sir_program == "flex":
        num_clusters = 55
    elif sir_program == "grep":
        num_clusters = 105
    elif sir_program == "gzip":
        num_clusters = 195
    elif sir_program == "sed":
        num_clusters = 195
    
    # Step 2: Perform K-Means Clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=43)
    start = time.time()
    clusters = kmeans.fit_predict(normalized_data)
    
    # Organize test cases by cluster
    clustered_data = defaultdict(list)
    for idx, cluster_id in enumerate(clusters):
        clustered_data[cluster_id].append(idx)
        
    # Step 4: Reassign points to ensure each cluster has at most 30 elements
    max_elements_per_cluster = 30
    for cluster_id, points in list(clustered_data.items()):
        if len(points) > max_elements_per_cluster:
            excess_points = points[max_elements_per_cluster:]
            clustered_data[cluster_id] = points[:max_elements_per_cluster]
            
            # Reassign excess points
            excess_data = normalized_data[excess_points]
            remaining_clusters = [k for k in clustered_data if len(clustered_data[k]) < max_elements_per_cluster]
            
            # Find nearest cluster with space for each excess point
            distances = cdist(excess_data, kmeans.cluster_centers_[remaining_clusters])
            nearest_clusters = np.argmin(distances, axis=1)
            
            for i, excess_idx in enumerate(excess_points):
                new_cluster = remaining_clusters[nearest_clusters[i]]
                clustered_data[new_cluster].append(excess_idx)
    
    end = time.time()
    print("SelectQAOA Decomposition Time (ms): " + str((end-start)*1000))
    
    clusters_dictionary[sir_program] = clustered_data
        
    # Step 3: Calculate the metrics for each cluster and validate
    cluster_metrics = {}
    for cluster_id in clustered_data.keys():
        tot_cluster_exec_cost = 0
        tot_cluster_past_fault_cov = 0
        tot_cluster_stmt_cov = 0
        for test_case in clustered_data[cluster_id]:
            tot_cluster_exec_cost += test_cases_costs[sir_program][test_case]
            tot_cluster_past_fault_cov += faults_dictionary[sir_program][test_case]
        tot_cluster_past_fault_cov = tot_cluster_past_fault_cov/tot_test_cases
        tot_cluster_stmt_cov = num_of_covered_lines(sir_program,clustered_data[cluster_id])/total_program_lines[sir_program]
        cluster_metrics[cluster_id] = {
            "tot_exec_cost": tot_cluster_exec_cost,
            "tot_past_fault_cov": tot_cluster_past_fault_cov,
            "tot_stmt_cov": tot_cluster_stmt_cov  # Avg stmt coverage per test case in cluster
        }
        print(f"Cluster {cluster_id + 1} metrics:")
        print(f"Test Cases: {clustered_data[cluster_id]}")
        print(f" - Num. Test Cases: {len(clustered_data[cluster_id]):.2f}")
        print(f" - Execution Cost: {tot_cluster_exec_cost:.2f}")
        print(f" - Past Fault Coverage (%): {tot_cluster_past_fault_cov}")
        print(f" - Statement Coverage (%): {tot_cluster_stmt_cov:.2f}\n")
    
    for cluster_id in clustered_data.keys():
        print("Test cases of cluster " + str(cluster_id) + ": " + str(len(clustered_data[cluster_id])))
    
    print("======================================================================================")
    
    print("Program Name: " + sir_program)
    
    for cluster_id in clustered_data.keys():
        if len(clustered_data[cluster_id]) > max_elements_per_cluster:
            print("Test cases of cluster " + str(cluster_id) + ": " + str(len(clustered_data[cluster_id])))
    
    # Plotting the clusters in 3D space
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    
    # Extracting data for plotting
    exec_costs = np.array(list(test_cases_costs[sir_program].values()))
    fault_covs = np.array(faults_dictionary[sir_program])
    stmt_covs = np.array(test_cases_stmt_cov)
    
    # Plot each cluster with a different color
    colors = plt.cm.get_cmap("tab10", num_clusters)  # A colormap with as many colors as clusters
    for cluster_id in clustered_data.keys():
        cluster_indices = clustered_data[cluster_id]
        
        # Plot each cluster's points
        ax.scatter(
            exec_costs[cluster_indices], 
            fault_covs[cluster_indices], 
            stmt_covs[cluster_indices], 
            color=colors(cluster_id), 
            label=f"Cluster {cluster_id + 1}"
        )
    
    # Label the axes
    ax.set_xlabel("Execution Cost")
    ax.set_ylabel("Past Fault Coverage")
    ax.set_zlabel("Statement Coverage")
    ax.legend()
    ax.set_title("Test Case Clustering Visualization")
    
    # Display the plot
    plt.show()
    
print(clusters_dictionary)

In [ ]:
def make_linear_terms(sir_program, cluster_test_cases, alpha):
    max_cost = max(test_cases_costs[sir_program].values())
    
    estimated_costs = []

    #linear coefficients, that are the diagonal of the matrix encoding the QUBO
    for test_case in cluster_test_cases:
        estimated_costs.append((alpha * (test_cases_costs[sir_program][test_case]/max_cost)) - (1 - alpha) * faults_dictionary[sir_program][test_case])
    
    return np.array(estimated_costs)

def make_quadratic_terms(sir_program, variables, cluster_test_cases, linear_terms, penalty):
    quadratic_terms = {}
    
    #k is a stmt
    for k in executed_lines_test_by_test[sir_program].keys():
        #k_test_cases is the list of test cases covering k
        k_test_cases = executed_lines_test_by_test[sir_program][k]
        for i in k_test_cases:
            if i not in cluster_test_cases or i not in variables:
                continue
            for j in k_test_cases:
                if j not in cluster_test_cases or j not in variables:
                    continue
                if i < j:
                    linear_terms[variables.index(i)] -= penalty
                    try:
                        quadratic_terms[variables.index(i),variables.index(j)] += 2 * penalty
                    except:
                        quadratic_terms[variables.index(i),variables.index(j)] = 2 * penalty
    
    return quadratic_terms

In [ ]:
def create_QUBO_problem(linear_terms, quadratic_terms):
    """This function is the one that has to encode the QUBO problem that QAOA will have to solve. The QUBO problem specifies the optimization to solve and a quadratic binary unconstrained problem"""
    qubo = QuadraticProgram()
    
    for i in range(0,len(linear_terms)):
        qubo.binary_var('x%s' % (i))

    qubo.minimize(linear=linear_terms,quadratic=quadratic_terms)

    return qubo


In [ ]:
penalties_dictionary = {"flex":None,"grep":None,"gzip":None,"sed":None}

#to obtain a QUBO problem from a quadratic problem with constraints, we have to insert those constraints into the Hamiltonian to solve (which is the one encoded by the QUBO problem). When we insert constraint into the Hamiltonian, we have to specify also penalties
for sir_program in sir_programs:
    max_penalty = 0
    max_cost = max(test_cases_costs[sir_program].values())
    for i in range(sir_programs_tests_number[sir_program]):
        cost = (alpha * (test_cases_costs[sir_program][i]/max_cost)) - ((1 - alpha) * faults_dictionary[sir_program][i])
        if cost > max_penalty:
            max_penalty = cost
    penalties_dictionary[sir_program] = max_penalty + 1

In [ ]:
qubos_dictionary = {"flex":[],"grep":[],"gzip":[],"sed":[]}
#make a dictionary that saves, for each program, the correspondent QUBO
for sir_program in sir_programs:
    print("SIR Program:\n")
    for cluster_id in clusters_dictionary[sir_program]:
        print("Cluster ID: " + str(cluster_id))
        variables = []
        for idx in range(0, len(clusters_dictionary[sir_program][cluster_id])):
            variables.append(idx)
        linear_terms = make_linear_terms(sir_program, clusters_dictionary[sir_program][cluster_id], alpha)
        quadratic_terms = make_quadratic_terms(sir_program, variables, clusters_dictionary[sir_program][cluster_id], linear_terms, penalties_dictionary[sir_program])
        qubo = create_QUBO_problem(linear_terms, quadratic_terms)
        qubos_dictionary[sir_program].append(qubo)
        print(qubo)
        print("/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/")
    print("======================================================================================")

In [20]:
def covered_lines(sir_program,test_cases_list):
    covered_lines = set()
    
    for test_case in test_cases_list:
        try:
            for covered_line in test_coverage_line_by_line[sir_program][test_case]:
                covered_lines.add(covered_line)
        except:
            continue
    
    return len(covered_lines)

def build_pareto_front(sir_program,selected_tests):
    pareto_front = []
    max_fault_coverage = 0
    max_stmt_coverage = 0
    
    for index in range(1,len(selected_tests)+1):
        #exract the first index selected tests
        candidate_solution = selected_tests[:index]
        candidate_solution_fault_coverage = 0
        candidate_solution_stmt_coverage = 0
        for selected_test in candidate_solution:
            candidate_solution_fault_coverage += faults_dictionary[sir_program][selected_test]
            candidate_solution_stmt_coverage += covered_lines(sir_program,candidate_solution)
        #if the actual pareto front dominates the candidate solution, get to the next candidate
        if max_fault_coverage >= candidate_solution_fault_coverage and max_stmt_coverage >= candidate_solution_stmt_coverage:
            continue
        #eventually update the pareto front information
        if candidate_solution_stmt_coverage > max_stmt_coverage:
            max_stmt_coverage = candidate_solution_stmt_coverage
        if candidate_solution_fault_coverage > max_fault_coverage:
            max_fault_coverage = candidate_solution_fault_coverage
        #add the candidate solution to the pareto front
        pareto_front.append(candidate_solution)
    
    return pareto_front

## Statevector Simulator

In [ ]:
algorithm_globals.random_seed = 10598
backend = Aer.get_backend("statevector_simulator")
qaoa_mes = QAOA(sampler=BackendSampler(backend=backend), optimizer=COBYLA(100), reps=1)
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA

#I want to run the sampler 30 times to obtain different results for each sir program
for sir_program in sir_programs:
    #the fronts will be saved into files
    print("SIR Program: " + sir_program)
    file_path = "results/selectqaoa/statevector_sim/" + sir_program + "-data.json"
    json_data = {}
    response = None
    qpu_run_times = []
    pareto_fronts_building_times = []
    for i in range(experiments):
        final_selected_tests = []
        cluster_dict_index = 0
        for qubo in qubos_dictionary[sir_program]:
            print("QUBO Problem: " + str(qubo) + "\n Cluster Number: " + str(cluster_dict_index))
            print("Cluster's Test Cases: " +str(list(clusters_dictionary[sir_program].values())[cluster_dict_index]))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(qubo)
            print("RESULTS: " + str(qaoa_result))
            e = time.time()
            qpu_run_times.append((e - s) * 1000)
            #let's extract the selected tests
            variable_values = qaoa_result.x
            indexes_selected_tests = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_tests))
            selected_tests = []
            for index in indexes_selected_tests:
                selected_tests.append(list(clusters_dictionary[sir_program].values())[cluster_dict_index][index])
            print("Selected tests: " + str(selected_tests))
            print("Experiment Number: " + str(i))
            cluster_dict_index += 1
            for selected_test in selected_tests:
                if selected_test not in final_selected_tests:
                    final_selected_tests.append(selected_test)
        i+=1
        #now we have to build the pareto front
        print("Final Selected Test Cases: " + str(final_selected_tests))
        print("Length of the final list of selected test cases: " + str(len(final_selected_tests)))
        start = time.time()
        pareto_front = build_pareto_front(sir_program, final_selected_tests)
        end = time.time()
        json_data["pareto_front_" + str(i)] = pareto_front
        pareto_front_building_time = (end - start) * 1000
        pareto_fronts_building_times.append(pareto_front_building_time)

    #compute the average time needed for the construction of a pareto frontier and run time
    mean_qpu_run_time = statistics.mean(qpu_run_times)
    mean_pareto_fronts_building_time = statistics.mean(pareto_fronts_building_times)
    json_data["mean_qpu_run_time(ms)"] = mean_qpu_run_time
    json_data["stdev_qpu_run_time(ms)"] = statistics.stdev(qpu_run_times)
    json_data["all_qpu_run_times(ms)"] = qpu_run_times
    json_data["mean_pareto_fronts_building_time(ms)"] = mean_pareto_fronts_building_time

    with open(file_path, "w") as file:
        json.dump(json_data, file)


## Ideal Aer Simulator

### QAOA Optimal Depth Analysis
The first use of QAOA with aer simulator backend is to spot its optimal depth value. To do so, we run 30 times (for each SIR program) QAOA for each of the chosen reps values: 1, 2, 4, 8, 16. Then, using the Kruskal-Wallis H-test, we statistically analyze if there are relevant differences in the results. We use QAOA with aer simulator to obtain an Optimal Ideal configuration of QAOA for solving the TCS problem.

In [ ]:
#I want to run the sampler 30 times to get different results for each sir program
for sir_program in sir_programs:
    reps = [1,2,4,8,16]
    for rep in reps:
        sim_ideal = AerSimulator()
        algorithm_globals.random_seed = 10598
        qaoa_mes = QAOA(sampler=BackendSampler(backend=sim_ideal), optimizer=COBYLA(100), reps=rep)
        qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
        #the fronts will be saved into files
        print("SIR Program: " + sir_program)
        file_path = "results/selectqaoa/ideal/" + sir_program + "-data-rep-" + str(rep) + ".json"
        json_data = {}
        response = None
        qpu_run_times = []
        pareto_fronts_building_times = []
        for i in range(experiments):
            final_selected_tests = []
            cluster_dict_index = 0
            for qubo in qubos_dictionary[sir_program]:
                print("QUBO Problem: " + str(qubo) + "\n Cluster Number: " + str(cluster_dict_index))
                print("Cluster's Test Cases: " +str(list(clusters_dictionary[sir_program].values())[cluster_dict_index]))
                #for each iteration get the result
                s = time.time()
                qaoa_result = qaoa.solve(qubo)
                print("RESULTS: " + str(qaoa_result))
                e = time.time()
                qpu_run_times.append((e - s) * 1000)
                #let's extract the selected tests
                variable_values = qaoa_result.x
                indexes_selected_tests = [index for index, value in enumerate(variable_values) if value == 1]
                print("Indexes of selected tests to convert. " + str(indexes_selected_tests))
                selected_tests = []
                for index in indexes_selected_tests:
                    selected_tests.append(list(clusters_dictionary[sir_program].values())[cluster_dict_index][index])
                print("Selected tests: " + str(selected_tests))
                print("Experiment Number: " + str(i))
                cluster_dict_index += 1
                for selected_test in selected_tests:
                    if selected_test not in final_selected_tests:
                        final_selected_tests.append(selected_test)
            i+=1
            #now we have to build the pareto front
            print("Final Selected Test Cases: " + str(final_selected_tests))
            print("Length of the final list of selected test cases: " + str(len(final_selected_tests)))
            start = time.time()
            pareto_front = build_pareto_front(sir_program, final_selected_tests)
            end = time.time()
            json_data["pareto_front_" + str(i)] = pareto_front
            pareto_front_building_time = (end - start) * 1000
            pareto_fronts_building_times.append(pareto_front_building_time)

    #compute the average time needed for the construction of a pareto frontier and run time
    mean_qpu_run_time = statistics.mean(qpu_run_times)
    mean_pareto_fronts_building_time = statistics.mean(pareto_fronts_building_times)
    json_data["mean_qpu_run_time(ms)"] = mean_qpu_run_time
    json_data["stdev_qpu_run_time(ms)"] = statistics.stdev(qpu_run_times)
    json_data["all_qpu_run_times(ms)"] = qpu_run_times
    json_data["mean_pareto_fronts_building_time(ms)"] = mean_pareto_fronts_building_time

    with open(file_path, "w") as file:
        json.dump(json_data, file)


/var/folders/kn/kd67nwcs41113fgwjn39d7fm0000gn/T/ipykernel_9939/403574147.py:7: DeprecationWarning: The class ``qiskit.primitives.backend_sampler.BackendSampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `BackendSampler` class is `BackendSamplerV2`.
  qaoa_mes = QAOA(sampler=BackendSampler(backend=sim_ideal), optimizer=COBYLA(100), reps=rep)


SIR Program: flex
QUBO Problem: minimize 70.38563106303513*x0*x2 - 35.1928149508474*x0 + 4.6791867798879533e-07*x1 + 2.311405517775977e-07*x2 + 3.157041682815968e-07*x3 + 4.735562524223953e-07*x4 + 3.833550614847962e-07*x5 + 3.833550614847962e-07*x6 (7 variables, 0 constraints, '')
 Cluster Number: 0
Cluster's Test Cases: [0, 2, 11, 542, 551, 552, 553]
RESULTS: fval=-35.1928149508474, x0=1.0, x1=0.0, x2=0.0, x3=0.0, x4=0.0, x5=0.0, x6=0.0, status=SUCCESS
Indexes of selected tests to convert. [0]
Selected tests: [0]
This Was The Experiment Number: 0
QUBO Problem: minimize -0.4986518924632677*x0 - 0.49865199393960746*x1 - 0.49865195447658645*x2 - 0.49863967583947005*x3 - 0.49863963637644904*x4 - 0.49863970402734226*x5 - 0.4986396645643212*x6 - 0.498639895704873*x7 (8 variables, 0 constraints, '')
 Cluster Number: 1
Cluster's Test Cases: [1, 204, 244, 329, 369, 459, 499, 519]
RESULTS: fval=-3.9891544173919176, x0=1.0, x1=1.0, x2=1.0, x3=1.0, x4=1.0, x5=1.0, x6=1.0, x7=1.0, status=SUCCESS


## The Kruskal-Wallis H-test
The Kruskal-Wallis H-test is a non-parametric statistical test used to determine whether there are statistically significant differences between the medians of three or more independent groups. It is an extension of the Mann-Whitney U test for more than two groups and is used when the assumptions of ANOVA (e.g., normality and homogeneity of variance) are not met. The test ranks all data points across groups, calculates the sum of ranks for each group, and determines whether these ranks differ significantly. It outputs the H statistic and a p-value, where a small p-value indicates significant differences between groups.

In [ ]:
flex_non_dom_solutions = []
grep_non_dom_solutions = []
gzip_non_dom_solutions = []
sed_non_dom_solutions = []

## Fake Vigo Noise Simulator

In [ ]:
device_backend = FakeVigoV2()

algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=BackendSampler(backend=device_backend), optimizer=COBYLA(100), reps=1)
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA

#I want to run the sampler 30 times to obtain different results for each sir program
for sir_program in sir_programs:
    #the fronts will be saved into files
    print(sir_program)
    file_path = "results/selectqaoa/fake_vigo/" + sir_program + "-data.json"
    json_data = {}
    response = None
    qpu_run_times = []
    pareto_fronts_building_times = []
    for i in range(experiments):
        final_selected_tests = []
        cluster_dict_index = 0
        for qubo in qubos_dictionary[sir_program]:
            print("Experiment Number: " + str(i))
            print("QUBO Problem: " + str(qubo) + "\nNumber: " + str(cluster_dict_index))
            print("Cluster's Test Cases: " +str(list(clusters_dictionary[sir_program].values())[cluster_dict_index]))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(qubo)
            print("RESULTS: " + str(qaoa_result))
            e = time.time()
            qpu_run_times.append((e - s) * 1000)
            #let's extract the selected tests
            variable_values = qaoa_result.x
            indexes_selected_tests = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_tests))
            selected_tests = []
            for index in indexes_selected_tests:
                selected_tests.append(list(clusters_dictionary[sir_program].values())[cluster_dict_index][index])
            print("Selected tests: " + str(selected_tests))
            cluster_dict_index += 1
            for selected_test in selected_tests:
                if selected_test not in final_selected_tests:
                    final_selected_tests.append(selected_test)
        i+=1
        #now we have to build the pareto front
        print("Final Selected Test Cases: " + str(final_selected_tests))
        print(len(final_selected_tests))
        start = time.time()
        pareto_front = build_pareto_front(sir_program, final_selected_tests)
        end = time.time()
        json_data["pareto_front_" + str(i)] = pareto_front
        pareto_front_building_time = (end - start) * 1000
        pareto_fronts_building_times.append(pareto_front_building_time)

    #compute the average time needed for the construction of a pareto frontier and run time
    mean_qpu_run_time = statistics.mean(qpu_run_times)
    mean_pareto_fronts_building_time = statistics.mean(pareto_fronts_building_times)
    json_data["mean_qpu_run_time(ms)"] = mean_qpu_run_time
    json_data["stdev_qpu_run_time(ms)"] = statistics.stdev(qpu_run_times)
    json_data["all_qpu_run_times(ms)"] = qpu_run_times
    json_data["mean_pareto_fronts_building_time(ms)"] = mean_pareto_fronts_building_time

    with open(file_path, "w") as file:
        json.dump(json_data, file)


## Depolarizing Error Simulator

In [ ]:
noise_dep = NoiseModel()

# Define a 1% depolarizing error for single-qubit and two-qubit gates
depolarizing_error_1q = depolarizing_error(0.01, 1)  # 1% depolarizing error for single-qubit gates
depolarizing_error_2q = depolarizing_error(0.01, 2)  # 1% depolarizing error for two-qubit gates

# Add the 1% depolarizing noise to the single-qubit gates (rx and rz)
noise_dep.add_all_qubit_quantum_error(depolarizing_error_1q, ['rx', 'rz'])

# Add the 1% depolarizing noise to the two-qubit gate (cx)
noise_dep.add_all_qubit_quantum_error(depolarizing_error_2q, ['cx'])

sim_dep = AerSimulator(noise_model=noise_dep)

algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=BackendSampler(backend=sim_dep), optimizer=COBYLA(100), reps=1)
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA

#I want to run the sampler 30 times to obtain different results for each sir program
for sir_program in sir_programs:
    #the fronts will be saved into files
    print(sir_program)
    file_path = "results/selectqaoa/depolarizing_sim/01/" + sir_program + "-data.json"
    json_data = {}
    response = None
    qpu_run_times = []
    pareto_fronts_building_times = []
    for i in range(experiments):
        final_selected_tests = []
        cluster_dict_index = 0
        for qubo in qubos_dictionary[sir_program]:
            print("Experiment Number: " + str(i))
            print("QUBO Problem: " + str(qubo) + "\nNumber: " + str(cluster_dict_index))
            print("Cluster's Test Cases: " +str(list(clusters_dictionary[sir_program].values())[cluster_dict_index]))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(qubo)
            print("RESULTS: " + str(qaoa_result))
            e = time.time()
            qpu_run_times.append((e - s) * 1000)
            #let's extract the selected tests
            variable_values = qaoa_result.x
            indexes_selected_tests = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_tests))
            selected_tests = []
            for index in indexes_selected_tests:
                selected_tests.append(list(clusters_dictionary[sir_program].values())[cluster_dict_index][index])
            print("Selected tests: " + str(selected_tests))
            cluster_dict_index += 1
            for selected_test in selected_tests:
                if selected_test not in final_selected_tests:
                    final_selected_tests.append(selected_test)
        i+=1
        #now we have to build the pareto front
        print("Final Selected Test Cases: " + str(final_selected_tests))
        print(len(final_selected_tests))
        start = time.time()
        pareto_front = build_pareto_front(sir_program, final_selected_tests)
        end = time.time()
        json_data["pareto_front_" + str(i)] = pareto_front
        pareto_front_building_time = (end - start) * 1000
        pareto_fronts_building_times.append(pareto_front_building_time)

    #compute the average time needed for the construction of a pareto frontier and run time
    mean_qpu_run_time = statistics.mean(qpu_run_times)
    mean_pareto_fronts_building_time = statistics.mean(pareto_fronts_building_times)
    json_data["mean_qpu_run_time(ms)"] = mean_qpu_run_time
    json_data["stdev_qpu_run_time(ms)"] = statistics.stdev(qpu_run_times)
    json_data["all_qpu_run_times(ms)"] = qpu_run_times
    json_data["mean_pareto_fronts_building_time(ms)"] = mean_pareto_fronts_building_time

    with open(file_path, "w") as file:
        json.dump(json_data, file)


In [ ]:
noise_dep = NoiseModel()

# Define a 1% depolarizing error for single-qubit and two-qubit gates
depolarizing_error_1q = depolarizing_error(0.02, 1)  # 1% depolarizing error for single-qubit gates
depolarizing_error_2q = depolarizing_error(0.02, 2)  # 1% depolarizing error for two-qubit gates

# Add the 1% depolarizing noise to the single-qubit gates (rx and rz)
noise_dep.add_all_qubit_quantum_error(depolarizing_error_1q, ['rx', 'rz'])

# Add the 1% depolarizing noise to the two-qubit gate (cx)
noise_dep.add_all_qubit_quantum_error(depolarizing_error_2q, ['cx'])

sim_dep = AerSimulator(noise_model=noise_dep)

algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=BackendSampler(backend=sim_dep), optimizer=COBYLA(100), reps=1)
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA

#I want to run the sampler 30 times to obtain different results for each sir program
for sir_program in sir_programs:
    #the fronts will be saved into files
    print(sir_program)
    file_path = "results/selectqaoa/depolarizing_sim/02/" + sir_program + "-data.json"
    json_data = {}
    response = None
    qpu_run_times = []
    pareto_fronts_building_times = []
    for i in range(experiments):
        final_selected_tests = []
        cluster_dict_index = 0
        for qubo in qubos_dictionary[sir_program]:
            print("Experiment Number: " + str(i))
            print("QUBO Problem: " + str(qubo) + "\nNumber: " + str(cluster_dict_index))
            print("Cluster's Test Cases: " +str(list(clusters_dictionary[sir_program].values())[cluster_dict_index]))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(qubo)
            print("RESULTS: " + str(qaoa_result))
            e = time.time()
            qpu_run_times.append((e - s) * 1000)
            #let's extract the selected tests
            variable_values = qaoa_result.x
            indexes_selected_tests = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_tests))
            selected_tests = []
            for index in indexes_selected_tests:
                selected_tests.append(list(clusters_dictionary[sir_program].values())[cluster_dict_index][index])
            print("Selected tests: " + str(selected_tests))
            cluster_dict_index += 1
            for selected_test in selected_tests:
                if selected_test not in final_selected_tests:
                    final_selected_tests.append(selected_test)
        i+=1
        #now we have to build the pareto front
        print("Final Selected Test Cases: " + str(final_selected_tests))
        print(len(final_selected_tests))
        start = time.time()
        pareto_front = build_pareto_front(sir_program, final_selected_tests)
        end = time.time()
        json_data["pareto_front_" + str(i)] = pareto_front
        pareto_front_building_time = (end - start) * 1000
        pareto_fronts_building_times.append(pareto_front_building_time)

    #compute the average time needed for the construction of a pareto frontier and run time
    mean_qpu_run_time = statistics.mean(qpu_run_times)
    mean_pareto_fronts_building_time = statistics.mean(pareto_fronts_building_times)
    json_data["mean_qpu_run_time(ms)"] = mean_qpu_run_time
    json_data["stdev_qpu_run_time(ms)"] = statistics.stdev(qpu_run_times)
    json_data["all_qpu_run_times(ms)"] = qpu_run_times
    json_data["mean_pareto_fronts_building_time(ms)"] = mean_pareto_fronts_building_time

    with open(file_path, "w") as file:
        json.dump(json_data, file)


In [ ]:
noise_dep = NoiseModel()

# Define a 1% depolarizing error for single-qubit and two-qubit gates
depolarizing_error_1q = depolarizing_error(0.05, 1)  # 1% depolarizing error for single-qubit gates
depolarizing_error_2q = depolarizing_error(0.05, 2)  # 1% depolarizing error for two-qubit gates

# Add the 1% depolarizing noise to the single-qubit gates (rx and rz)
noise_dep.add_all_qubit_quantum_error(depolarizing_error_1q, ['rx', 'rz'])

# Add the 1% depolarizing noise to the two-qubit gate (cx)
noise_dep.add_all_qubit_quantum_error(depolarizing_error_2q, ['cx'])

sim_dep = AerSimulator(noise_model=noise_dep)

algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=BackendSampler(backend=sim_dep), optimizer=COBYLA(100), reps=1)
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA

#I want to run the sampler 30 times to obtain different results for each sir program
for sir_program in sir_programs:
    #the fronts will be saved into files
    print(sir_program)
    file_path = "results/selectqaoa/depolarizing_sim/05/" + sir_program + "-data.json"
    json_data = {}
    response = None
    qpu_run_times = []
    pareto_fronts_building_times = []
    for i in range(experiments):
        final_selected_tests = []
        cluster_dict_index = 0
        for qubo in qubos_dictionary[sir_program]:
            print("Experiment Number: " + str(i))
            print("QUBO Problem: " + str(qubo) + "\nNumber: " + str(cluster_dict_index))
            print("Cluster's Test Cases: " +str(list(clusters_dictionary[sir_program].values())[cluster_dict_index]))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(qubo)
            print("RESULTS: " + str(qaoa_result))
            e = time.time()
            qpu_run_times.append((e - s) * 1000)
            #let's extract the selected tests
            variable_values = qaoa_result.x
            indexes_selected_tests = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_tests))
            selected_tests = []
            for index in indexes_selected_tests:
                selected_tests.append(list(clusters_dictionary[sir_program].values())[cluster_dict_index][index])
            print("Selected tests: " + str(selected_tests))
            cluster_dict_index += 1
            for selected_test in selected_tests:
                if selected_test not in final_selected_tests:
                    final_selected_tests.append(selected_test)
        i+=1
        #now we have to build the pareto front
        print("Final Selected Test Cases: " + str(final_selected_tests))
        print(len(final_selected_tests))
        start = time.time()
        pareto_front = build_pareto_front(sir_program, final_selected_tests)
        end = time.time()
        json_data["pareto_front_" + str(i)] = pareto_front
        pareto_front_building_time = (end - start) * 1000
        pareto_fronts_building_times.append(pareto_front_building_time)

    #compute the average time needed for the construction of a pareto frontier and run time
    mean_qpu_run_time = statistics.mean(qpu_run_times)
    mean_pareto_fronts_building_time = statistics.mean(pareto_fronts_building_times)
    json_data["mean_qpu_run_time(ms)"] = mean_qpu_run_time
    json_data["stdev_qpu_run_time(ms)"] = statistics.stdev(qpu_run_times)
    json_data["all_qpu_run_times(ms)"] = qpu_run_times
    json_data["mean_pareto_fronts_building_time(ms)"] = mean_pareto_fronts_building_time

    with open(file_path, "w") as file:
        json.dump(json_data, file)


## Empirical Evaluations
From the pareto fronts obtained on 10 different repetitions (for each programme under examination), the non-dominated samples were extrapolated. In this way, for each repetition of each program, a reference front was constructed to allow comparison between the algorithms: quantum annealing, additional greedy and DIV-GA.
For each algorithm, divided by programmes, the average number (and std. dev.) of samples in their pareto fronts was calculated. Likewise, the average number (and std. dev.) of samples from their own front present in the reference front was also calculated.
The latter metric was subjected to the Mann-Whitney's statistical U test and then to the Vargha-Delaney's $\hat{A}_{12}$ effect size calculation to establish its validity and statistical magnitude.

The four different algorithms were compared also in terms of Hypervolume. This metric quantify the effectiveness of Pareto frontiers comparing them against an ideal one. The closer they are to the ideal frontier, the more effective they are in terms, in this implementation, of percentage of faults detection loss.

Also, the execution times of the three different algorithms were stored and confronted. Since the classical and quantum components of the D-Wave's hybrid solver work synchronously, the only metric available for such a comparison was the total "run-time".

In [ ]:
#the other sizes are uniform
#this static information can be easily obtained from the pareto .json files

qtcs_flex_pareto_sizes = [187,187,187,187,187,187,187,187,187,187]
divga_flex_pareto_sizes = [140,140,140,140,140,140,140,140,140,140]
qaoa_statevector_flex_pareto_sizes = [447,434,413,409,434,455,446,410,416,427]
qaoa_ideal_flex_pareto_sizes = [517,515,518,523,516,518,505,514,518,526]
qaoa_fakevigo_flex_pareto_sizes = [540,540,540,540,540,540,540,539,539,539]
qaoa_noise1_flex_pareto_sizes = [539,540,540,539,540,540,539,539,539,539]
qaoa_noise2_flex_pareto_sizes = [540,539,540,539,540,540,539,540,539,540]
qaoa_noise5_flex_pareto_sizes = [539,539,540,540,540,539,540,540,540,539]
greedy_flex_size = 567

qtcs_grep_pareto_sizes = [229,229,230,229,230,229,230.229,230,230]
divga_grep_pareto_sizes = [70,70,70,70,70,70,70,70,70,70]
qaoa_statevector_grep_pareto_sizes = [485,507,504,505,489,492,489,493,489,470]
qaoa_ideal_grep_pareto_sizes = [343,345,343,341,346,345,345,346,340,348]
qaoa_fakevigo_grep_pareto_sizes = [321,321,321,321,321,321,321,321,321,321]
qaoa_noise1_grep_pareto_sizes = [321,321,321,321,321,321,321,321,321,321]
qaoa_noise2_grep_pareto_sizes = [321,321,321,321,321,321,321,321,321,321]
qaoa_noise5_grep_pareto_sizes = [321,321,321,321,321,321,321,321,321,321]
greedy_grep_size = 802

qtcs_gzip_pareto_sizes = [87,86,85,86,87,87,86,87,85,87]
divga_gzip_pareto_sizes = [105,105,105,105,105,105,105,105,105,105]
qaoa_statevector_gzip_pareto_sizes = [137,143,146,144,164,143,157,144,141,169]
qaoa_ideal_gzip_pareto_sizes = [147,146,145,146,151,155,155,151,154,156]
qaoa_fakevigo_gzip_pareto_sizes = [168,168,166,169,168,167,167,168,168,167]
qaoa_noise1_gzip_pareto_sizes = [167,167,167,166,168,167,166,166,168,166]
qaoa_noise2_gzip_pareto_sizes = [166,166,168,167,166,166,166,167,167,166]
qaoa_noise5_gzip_pareto_sizes = [169,167,167,169,168,168,168,165,167,167]
greedy_gzip_size = 199

qtcs_sed_pareto_sizes = [131,131,131,131,131,131,131,131,131,131]
divga_sed_pareto_sizes = [105,62,105,105,102,105,105,97,105,105]
qaoa_statevector_sed_pareto_sizes = [249,241,252,226,245,231,248,241,245,237]
qaoa_ideal_sed_pareto_sizes = [260,251,253,263,252,263,260,256,250,253]
qaoa_fakevigo_sed_pareto_sizes = [235,235,236,235,236,236,235,236,236,236]
qaoa_noise1_sed_pareto_sizes = [236,235,235,235,236,236,236,235,236,235]
qaoa_noise2_sed_pareto_sizes = [235,235,236,235,235,236,236,236,236,236]
qaoa_noise5_sed_pareto_sizes = [236,236,235,236,236,235,235,236,236,235]
greedy_sed_size = 356

In [ ]:
def total_cost(sir_program,solution):
    solution_cost = 0
    
    for test_case in solution:
        solution_cost += test_cases_costs[sir_program][test_case]

    return solution_cost
        
def total_coverage(sir_program,solution):
    covered_lines = set()
    
    for test_case in solution:
        try:
            for covered_line in test_coverage_line_by_line[sir_program][test_case]:
                covered_lines.add(covered_line)
        except:
            continue
    
    return len(covered_lines)
    
def total_faults(sir_program,solution):
    covered_faults = 0
    
    for test_case in solution:
        covered_faults += faults_dictionary[sir_program][test_case]
    
    return covered_faults

def pareto_dominance(tuple1, tuple2):
    # Check if all conditions are satisfied
    dominates = (
        (tuple2[0] <= tuple1[0]) and 
        (tuple2[1] >= tuple1[1]) and 
        (tuple2[2] >= tuple1[2])
    )
    
    # Check if at least one condition is strict
    strict = (
        (tuple2[0] < tuple1[0]) or 
        (tuple2[1] > tuple1[1]) or 
        (tuple2[2] > tuple1[2])
    )
    
    # Return 1 if the second tuple dominates the first, otherwise 0
    return 1 if dominates and strict else 0

In [ ]:
with open('./results/add-greedy/grep_data.json', 'r') as file:
    pareto_fronts_json = json.load(file)
    
greedy_pareto_front = pareto_fronts_json['pareto_front']

#each solution of the pareto front is a subset of the test suite
greedy_pareto_vectors = []
for front_solution in greedy_pareto_front:
    #compute the fitness scores of each single solution
    greedy_pareto_vectors.append((total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)))

In [ ]:
#here we make the same but for each of the fronts built by SelectQA and DIV-GA

#the following 2 lists will contain much tuples as solutions in each of the 10 pareto fronts
#each tuple represent the fitness value for each solution
qtcs_pareto_vectors = []
divga_pareto_vectors = []
qaoa_statevector_pareto_vectors = []
qaoa_ideal_pareto_vectors = []
qaoa_fakevigo_pareto_vectors = []
qaoa_noise1_pareto_vectors = []
qaoa_noise2_pareto_vectors = []
qaoa_noise5_pareto_vectors = []

for index in range(0,10):
    # Load the JSON file
    with open('results/selectqa/grep-data.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    qtcs_pareto_front = pareto_fronts_json['pareto_front_'+str(index)]
    #a single solution is a subset of the initial test suite
    for front_solution in qtcs_pareto_front:
        qtcs_pareto_vectors.append((total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)))
    #print(len(qtcs_pareto_vectors))
    
    with open('./results/divga/grep_pareto_fronts_divga.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    divga_pareto_front = pareto_fronts_json['grep_pareto_front_'+str(index)]
    for front_solution in divga_pareto_front:
        divga_pareto_vectors.append((total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)))
    #print(len(divga_pareto_vectors))
    
    with open('results/selectqaoa/statevector_sim/grep-data.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
    #a single solution is a subset of the initial test suite
    for front_solution in qaoa_pareto_front:
        qaoa_statevector_pareto_vectors.append((total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)))
    print(len(qaoa_statevector_pareto_vectors))
    
    with open('results/selectqaoa/ideal/grep-data.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
    #a single solution is a subset of the initial test suite
    for front_solution in qaoa_pareto_front:
        qaoa_ideal_pareto_vectors.append((total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)))
    #print(len(qaoa_ideal_pareto_vectors))
    
    with open('results/selectqaoa/fake_vigo/grep-data.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
    #a single solution is a subset of the initial test suite
    for front_solution in qaoa_pareto_front:
        qaoa_fakevigo_pareto_vectors.append((total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)))
    #print(len(qaoa_fakevigo_pareto_vectors))
    
    with open('results/selectqaoa/depolarizing_sim/01/grep-data.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
    #a single solution is a subset of the initial test suite
    for front_solution in qaoa_pareto_front:
        qaoa_noise1_pareto_vectors.append((total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)))
    #print(len(qaoa_noise1_pareto_vectors))
    
    with open('results/selectqaoa/depolarizing_sim/02/grep-data.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
    #a single solution is a subset of the initial test suite
    for front_solution in qaoa_pareto_front:
        qaoa_noise2_pareto_vectors.append((total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)))
    #print(len(qaoa_noise2_pareto_vectors))
    
    with open('results/selectqaoa/depolarizing_sim/05/grep-data.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
    #a single solution is a subset of the initial test suite
    for front_solution in qaoa_pareto_front:
        qaoa_noise5_pareto_vectors.append((total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)))
    #print(len(qaoa_noise5_pareto_vectors))

#once we have the pareto vectors from each pareto front obtained by each run, we extract 
#just the solution not dominated by anyone else
#qtcs_pareto_vectors,divga_pareto_vectors,greedy_pareto_vectors,qaoa_ideal_pareto_vectors,qaoa_fakevigo_pareto_vectors,qaoa_noise1_pareto_vectors,qaoa_noise2_pareto_vectors,qaoa_noise5_pareto_vectors
total_fronts = [qtcs_pareto_vectors,divga_pareto_vectors,greedy_pareto_vectors,qaoa_noise5_pareto_vectors]
reference_pareto = []
portions = [0,0,0,0]

# get the reference frontier
for index, front1 in enumerate(total_fronts):
    for front_solution1 in front1:
        is_dominated = 0
        other_fronts = total_fronts[:index] + total_fronts[index+1:]
        for front2 in other_fronts:
            for front_solution2 in front2:
                if pareto_dominance(front_solution1,front_solution2):
                    is_dominated = 1
                    break
            if is_dominated:
                break
        if not is_dominated:
            reference_pareto.append(front_solution1)
            portions[index] = portions[index] + 1

print(portions)

In [ ]:
# For SelectQA and DIV-GA we want to compute, for each of the 10 iterations, how many of the solutions of the i-th pareto front were selected by the reference front

qtcs_non_dominated_values = []
divga_non_dominated_values = []
qaoa_statevector_non_dominated_values = []
qaoa_ideal_non_dominated_values = []
qaoa_fakevigo_non_dominated_values = []
qaoa_noise1_non_dominated_values = []
qaoa_noise2_non_dominated_values = []
qaoa_noise5_non_dominated_values = []
greedy_non_dominated = 0

for index in range(0,10):
    #print("Iteratrion: " + str(index))
    
    qtcs_non_dominated = 0
    divga_non_dominated = 0
    qaoa_statevector_non_dominated = 0
    qaoa_ideal_non_dominated = 0
    qaoa_fakevigo_non_dominated = 0
    qaoa_noise1_non_dominated = 0
    qaoa_noise2_non_dominated = 0
    qaoa_noise5_non_dominated = 0
    
    # Load the JSON file
    with open('results/selectqa/grep-data.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    qtcs_pareto_front = pareto_fronts_json['pareto_front_'+str(index)]
    for front_solution in qtcs_pareto_front:
        if (total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)) in reference_pareto:
            qtcs_non_dominated += 1
    
    qtcs_non_dominated_values.append(qtcs_non_dominated)
    #print(qtcs_non_dominated)
    
    with open('./results/divga/grep_pareto_fronts_divga.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    divga_pareto_front = pareto_fronts_json['grep_pareto_front_'+str(index)]
    for front_solution in divga_pareto_front:
        if (total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)) in reference_pareto:
            divga_non_dominated += 1
            
    divga_non_dominated_values.append(divga_non_dominated)
    #print(divga_non_dominated)
    
    with open('results/selectqaoa/statevector_sim/grep-data.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
    for front_solution in qaoa_pareto_front:
        if (total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)) in reference_pareto:
            qaoa_statevector_non_dominated += 1
    
    qaoa_statevector_non_dominated_values.append(qaoa_statevector_non_dominated)
    print(qaoa_statevector_non_dominated)
    
    with open('results/selectqaoa/ideal/grep-data.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
    for front_solution in qaoa_pareto_front:
        if (total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)) in reference_pareto:
            qaoa_ideal_non_dominated += 1
    
    qaoa_ideal_non_dominated_values.append(qaoa_ideal_non_dominated)
    #print(qaoa_statevector_non_dominated)
    
    with open('results/selectqaoa/fake_vigo/grep-data.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
    for front_solution in qaoa_pareto_front:
        if (total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)) in reference_pareto:
            qaoa_fakevigo_non_dominated += 1
    
    qaoa_fakevigo_non_dominated_values.append(qaoa_fakevigo_non_dominated)
    #print(qaoa_statevector_non_dominated)
    
    with open('results/selectqaoa/depolarizing_sim/01/grep-data.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
    for front_solution in qaoa_pareto_front:
        if (total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)) in reference_pareto:
            qaoa_noise1_non_dominated += 1
    
    qaoa_noise1_non_dominated_values.append(qaoa_noise1_non_dominated)
    #print(qaoa_statevector_non_dominated)
    
    with open('results/selectqaoa/depolarizing_sim/02/grep-data.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
    for front_solution in qaoa_pareto_front:
        if (total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)) in reference_pareto:
            qaoa_noise2_non_dominated += 1
    
    qaoa_noise2_non_dominated_values.append(qaoa_noise2_non_dominated)
    #print(qaoa_statevector_non_dominated)
    
    with open('results/selectqaoa/depolarizing_sim/05/grep-data.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
    for front_solution in qaoa_pareto_front:
        if (total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)) in reference_pareto:
            qaoa_noise5_non_dominated += 1
    
    qaoa_noise5_non_dominated_values.append(qaoa_noise5_non_dominated)
    #print(qaoa_statevector_non_dominated)
    
for front_solution in greedy_pareto_front:
    if (total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)) in reference_pareto:
        greedy_non_dominated += 1
print(greedy_non_dominated)

print("QTCS/DIVGA/ADD GREEDY")
print("SelectQA Non Dominated Values")
print(qtcs_non_dominated_values)
print("SelectQA Non Dominated Mean")
print(statistics.mean(qtcs_non_dominated_values))
print("SelectQA Non Dominated StDev")
print(statistics.stdev(qtcs_non_dominated_values))

print("DIV-GA Non Dominated Values")
print(divga_non_dominated_values)
print("DIV-GA Non Dominated Mean")
print(statistics.mean(divga_non_dominated_values))
print("DIV-GA Non Dominated StDev")
print(statistics.stdev(divga_non_dominated_values))

print("SelectQAOA Non Dominated Values (Statevector)")
print(qtcs_non_dominated_values)
print("SelectQAOA Non Dominated Mean (Statevector)")
print(statistics.mean(qtcs_non_dominated_values))
print("SelectQAOA Non Dominated StDev (Statevector)")
print(statistics.stdev(qtcs_non_dominated_values))

print("SelectQAOA Non Dominated Values (Ideal)")
print(qaoa_ideal_non_dominated_values)
print("SelectQAOA Non Dominated Mean (Ideal)")
print(statistics.mean(qaoa_ideal_non_dominated_values))
print("SelectQAOA Non Dominated StDev (Ideal)")
print(statistics.stdev(qaoa_ideal_non_dominated_values))

print("SelectQAOA Non Dominated Values (Fake Vigo)")
print(qaoa_fakevigo_non_dominated_values)
print("SelectQAOA Non Dominated Mean (Fake Vigo)")
print(statistics.mean(qaoa_fakevigo_non_dominated_values))
print("SelectQAOA Non Dominated StDev (Fake Vigo)")
print(statistics.stdev(qaoa_fakevigo_non_dominated_values))

print("SelectQAOA Non Dominated Values (Depolarizing Error 1%)")
print(qaoa_noise1_non_dominated_values)
print("SelectQAOA Non Dominated Mean (Depolarizing Error 1%)")
print(statistics.mean(qaoa_noise1_non_dominated_values))
print("SelectQAOA Non Dominated StDev (Depolarizing Error 1%)")
print(statistics.stdev(qaoa_noise1_non_dominated_values))

print("SelectQAOA Non Dominated Values (Depolarizing Error 2%)")
print(qaoa_noise2_non_dominated_values)
print("SelectQAOA Non Dominated Mean (Depolarizing Error 2%)")
print(statistics.mean(qaoa_noise2_non_dominated_values))
print("SelectQAOA Non Dominated StDev (Depolarizing Error 2%)")
print(statistics.stdev(qaoa_noise2_non_dominated_values))

print("SelectQAOA Non Dominated Values (Depolarizing Error 5%)")
print(qaoa_noise5_non_dominated_values)
print("SelectQAOA Non Dominated Mean (Depolarizing Error 5%)")
print(statistics.mean(qaoa_noise5_non_dominated_values))
print("SelectQAOA Non Dominated StDev (Depolarizing Error 5%)")
print(statistics.stdev(qaoa_noise5_non_dominated_values))

print("Greedy Non Dominated Values")
print(greedy_non_dominated)

In [ ]:
# Unpack the tuples for each list
def unpack_tuples(data):
    return zip(*data)

#qtcs_pareto_vectors,divga_pareto_vectors,greedy_pareto_vectors

#x1, y1, z1 = unpack_tuples(qtcs_pareto_vectors)
#x2, y2, z2 = unpack_tuples(divga_pareto_vectors)
#x3, y3, z3 = unpack_tuples(greedy_pareto_vectors)
x4, y4, z4 = unpack_tuples(qaoa_noise5_pareto_vectors)
x5, y5, z5 = unpack_tuples(reference_pareto)

# Create 3D scatter plots
fig = plt.figure(figsize=(18,6))

"""# First plot: list1 vs list5
ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(x1, y1, z1, c='r', marker='s', s=100, label='QA')
ax1.scatter(x5, y5, z5, c='b', marker='o', s=50, label='Reference Pareto')
ax1.set_xlabel('Cost')
ax1.set_ylabel('Statement Coverage')
ax1.set_zlabel('Past Faults',labelpad=0.2)
ax1.set_title('QA vs Reference')
ax1.legend()

# Second plot: list2 vs list5
ax2 = fig.add_subplot(132, projection='3d')
ax2.scatter(x2, y2, z2, c='g', marker='s', s=100, label='DIV-GA')
ax2.scatter(x5, y5, z5, c='b', marker='o', s=50, label='Reference Pareto')
ax2.set_xlabel('Cost')
ax2.set_ylabel('Statement Coverage')
ax2.set_zlabel('Past Faults',labelpad=0.2)
ax2.set_title('DIV-GA vs Reference Pareto')
ax2.legend()

# Third plot: list3 vs list5
ax3 = fig.add_subplot(133, projection='3d')
ax3.scatter(x3, y3, z3, c='m', marker='s', s=100, label='Add. Greedy')
ax3.scatter(x5, y5, z5, c='b', marker='o', s=50, label='Reference Pareto')
ax3.set_xlabel('Cost')
ax3.set_ylabel('Statement Coverage')
ax3.set_zlabel('Past Faults',labelpad=0.2)
ax3.set_title('Add. Greedy vs Reference Pareto')
ax3.legend()"""

# Third plot: list4 vs list5
ax4 = fig.add_subplot(133, projection='3d')
ax4.scatter(x4, y4, z4, c='m', marker='s', s=100, label='QAOA')
ax4.scatter(x5, y5, z5, c='b', marker='o', s=50, label='Reference Pareto')
ax4.set_xlabel('Cost')
ax4.set_ylabel('Statement Coverage')
ax4.set_zlabel('Past Faults',labelpad=0.2)
ax4.set_title('Ideal QAOA vs Reference Pareto')
ax4.legend()

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
from scipy.interpolate import interp1d


def interpolate_curve(curve, num_points):
    """Interpolate a 3D curve to have a specified number of points."""
    curve = np.array(curve)
    t = np.linspace(0, 1, len(curve))
    interp_func = interp1d(t, curve, axis=0, kind='linear')
    t_new = np.linspace(0, 1, num_points)
    return interp_func(t_new)

def compute_area_between_curves(curve1, curve2):
    """Compute the approximate area between two 3D curves with different numbers of points."""
    # Determine the target number of points
    num_points = max(len(curve1), len(curve2))
    
    # Interpolate both curves to have the same number of points
    curve1_resampled = interpolate_curve(curve1, num_points)
    curve2_resampled = interpolate_curve(curve2, num_points)
    
    # Calculate the distance between corresponding points on the two curves
    distances = np.linalg.norm(curve1_resampled - curve2_resampled, axis=1)
    
    # Calculate the distance between consecutive points on the curves
    segment_lengths = np.linalg.norm(np.diff(curve1_resampled, axis=0), axis=1)
    
    # Compute the approximate area using the trapezoidal rule
    area = np.sum((distances[:-1] + distances[1:]) / 2 * segment_lengths)
    
    return area

area_between_curves = compute_area_between_curves(qtcs_pareto_vectors, reference_pareto)
print("Approximate area between the two curves:", area_between_curves)

In [ ]:
# greedy has just one frontier
#print("Greedy Pareto Vectors: " + str(greedy_pareto_vectors))
#print("DIV-GA Pareto Vectors: " + str(divga_pareto_vectors))
#print("SelectQA Pareto Vectors: " + str(qtcs_pareto_vectors))
#total_fronts has fronts, each front has different solutions
IH_add_greedy = greedy_pareto_vectors[0][0]
for index, solution in enumerate(greedy_pareto_vectors):
    if index == len(greedy_pareto_vectors) - 2:
        break
    IH_add_greedy += (greedy_pareto_vectors[index+1][0] - greedy_pareto_vectors[index][0]) * (1 - (greedy_pareto_vectors[index][2]/len(greedy_pareto_front[index])))
IEC_add_greedy = IH_add_greedy/greedy_pareto_vectors[len(greedy_pareto_vectors)-1][0]
print("IH Additional Greedy: " + str(IH_add_greedy))
print("IEC Additional Greedy: " + str(IEC_add_greedy))

In [ ]:
IECs_qtcs = []
IECs_divga = []
IECs_qaoa = []

for front_index in range(0,10):
    print("IT Number: " + str(front_index))
    IH_qtcs = 0
    IH_divga = 0
    IEC_qtcs = 0
    IEC_divga = 0
    IH_qaoa = 0
    IEC_qaoa = 0
    # Load the JSON file
    with open('results/selectqa/old/grep-data.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    qtcs_pareto_front = pareto_fronts_json['pareto_front_'+str(front_index)]
    qtcs_pareto_vectors = []
    #a single solution is a subset of the initial test suite
    for front_solution in qtcs_pareto_front:
        qtcs_pareto_vectors.append((total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)))
    #print(len(qtcs_pareto_vectors))
    
    for index, solution in enumerate(qtcs_pareto_vectors):
        if index == len(qtcs_pareto_vectors) - 2:
            break
        IH_qtcs += (qtcs_pareto_vectors[index+1][0] - qtcs_pareto_vectors[index][0]) * (1 - (qtcs_pareto_vectors[index][2]/len(qtcs_pareto_front[index])))
    IEC_qtcs = IH_qtcs/qtcs_pareto_vectors[len(qtcs_pareto_vectors)-1][0]
    IECs_qtcs.append(IEC_qtcs)
    qtcs_pareto_vectors = sorted(qtcs_pareto_vectors, key=lambda x: x[0])
    print("IH SelectQA: " + str(IH_qtcs))
    print("IEC SelectQA: " + str(IEC_qtcs))
    
    with open('./results/divga/grep_pareto_fronts_divga.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    divga_pareto_front = pareto_fronts_json['grep_pareto_front_'+str(front_index)]
    divga_pareto_vectors = []
    for front_solution in divga_pareto_front:
        divga_pareto_vectors.append((total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)))
    divga_pareto_vectors = sorted(divga_pareto_vectors, key=lambda x: x[0])
    #print(len(divga_pareto_vectors))
    
    for index, solution in enumerate(divga_pareto_vectors):
        if index == len(divga_pareto_vectors) - 2:
            break
        IH_divga += (divga_pareto_vectors[index+1][0] - divga_pareto_vectors[index][0]) * (1 - (divga_pareto_vectors[index][2]/len(faults_dictionary["grep"])))
    IEC_divga = IH_divga/divga_pareto_vectors[len(divga_pareto_vectors)-1][0]
    IECs_divga.append(IEC_divga)
    print("IH DIV-GA: " + str(IH_divga))
    print("IEC DIV-GA: " + str(IEC_divga))
    
    # Load the JSON file
    with open('results/selectqaoa/statevector_sim/grep-data.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(front_index+1)]
    qaoa_pareto_vectors = []
    #a single solution is a subset of the initial test suite
    for front_solution in qaoa_pareto_front:
        qaoa_pareto_vectors.append((total_cost("grep",front_solution),total_coverage("grep",front_solution),total_faults("grep",front_solution)))
    #print(len(qaoa_pareto_vectors))
    
    for index, solution in enumerate(qaoa_pareto_vectors):
        if index == len(qaoa_pareto_vectors) - 2:
            break
        IH_qaoa += (qaoa_pareto_vectors[index+1][0] - qaoa_pareto_vectors[index][0]) * (1 - (qaoa_pareto_vectors[index][2]/len(qaoa_pareto_front[index])))
    IEC_qaoa = IH_qaoa/qaoa_pareto_vectors[len(qaoa_pareto_vectors)-1][0]
    IECs_qaoa.append(IEC_qaoa)
    qaoa_pareto_vectors = sorted(qaoa_pareto_vectors, key=lambda x: x[0])
    print("IH SelectQAOA: " + str(IH_qaoa))
    print("IEC SelectQAOA: " + str(IEC_qaoa))
    
print("Mean IECs_qtcs: " + str(statistics.mean(IECs_qtcs)))
print("Mean IECs_divga: " + str(statistics.mean(IECs_divga)))
print("Mean IECs_qaoa: " + str(statistics.mean(IECs_qaoa)))

In [ ]:
"""
NON DOM SEQUENCES 
IDEAL
qtcs_nondom_flex = [187, 187, 187, 187, 187, 187, 187, 187, 187, 187]
div_ga_nondom_flex = [140, 140, 140, 140, 140, 140, 140, 140, 140, 139]
qaoa_nondom_flex = [471, 479, 479, 487, 478, 478, 468, 478, 473, 482]
add_greedy_nondom_flex = [205,205,205,205,205,205,205,205,205,205]

qtcs_nondom_grep = [10, 10, 11, 10, 11, 10, 11, 10, 11, 11]
div_ga_nondom_grep = [70, 70, 70, 70, 70, 70, 70, 70, 70, 70]
qaoa_nondom_grep = [376, 316, 370, 348, 327, 338, 350, 316, 358, 405]
add_greedy_nondom_grep = [171,171,171,171,171,171,171,171,171,171]

qtcs_nondom_gzip = [38, 37, 36, 37, 38, 38, 37, 38, 36, 38]
div_ga_nondom_gzip = [105, 105, 105, 105, 105, 105, 105, 105, 105, 105]
qaoa_nondom_gzip = [343, 345, 343, 341, 346, 345, 345, 346, 340, 348]
add_greedy_nondom_gzip = [33,33,33,33,33,33,33,33,33,33]

qtcs_nondom_sed = [110, 110, 110, 110, 110, 110, 110, 110, 110, 110]
div_ga_nondom_sed = [95, 62, 99, 105, 90, 100, 98, 93, 103, 104]
qaoa_nondom_sed = [192, 196, 198, 188, 194, 188, 196, 189, 195, 199]
add_greedy_nondom_sed = [80,80,80,80,80,80,80,80,80,80]

FAKE VIGO
qtcs_nondom_flex = [187, 187, 187, 187, 187, 187, 187, 187, 187, 187]
div_ga_nondom_flex = [140, 140, 140, 140, 140, 140, 140, 140, 140, 140]
qaoa_nondom_flex = [492, 492, 492, 492, 492, 492, 492, 491, 491, 491]
add_greedy_nondom_flex = [205,205,205,205,205,205,205,205,205,205]

qtcs_nondom_grep = [192, 192, 193, 192, 193, 192, 193, 192, 193, 193]
div_ga_nondom_grep = [70, 70, 70, 70, 70, 70, 70, 70, 70, 70]
qaoa_nondom_grep = [321, 321, 321, 321, 321, 321, 321, 321, 321, 321]
add_greedy_nondom_grep = [177,177,177,177,177,177,177,177,177,177]

qtcs_nondom_gzip = [41, 40, 39, 40, 41, 41, 40, 41, 39, 41]
div_ga_nondom_gzip = [105, 105, 105, 105, 105, 105, 105, 105, 105, 105]
qaoa_nondom_gzip = [95, 95, 103, 96, 95, 104, 104, 95, 95, 104]
add_greedy_nondom_gzip = [26,26,26,26,26,26,26,26,26,26]

qtcs_nondom_sed = [86, 86, 86, 86, 86, 86, 86, 86, 86, 86]
div_ga_nondom_sed = [89, 62, 103, 101, 89, 97, 97, 94, 101, 103]
qaoa_nondom_sed = [183, 183, 183, 183, 183, 183, 183, 183, 183, 183]
add_greedy_nondom_sed = [80,80,80,80,80,80,80,80,80,80]

NOISE 1%
qtcs_nondom_flex = [187, 187, 187, 187, 187, 187, 187, 187, 187, 187]
div_ga_nondom_flex = [140, 140, 140, 140, 140, 140, 140, 140, 140, 140]
qaoa_nondom_flex = [491, 492, 492, 491, 492, 492, 491, 491, 491, 491]
add_greedy_nondom_flex = [205,205,205,205,205,205,205,205,205,205]

qtcs_nondom_grep = [192, 192, 193, 192, 193, 192, 193, 192, 193, 193]
div_ga_nondom_grep = [70, 70, 70, 70, 70, 70, 70, 70, 70, 70]
qaoa_nondom_grep = [321, 321, 321, 321, 321, 321, 321, 321, 321, 321]
add_greedy_nondom_grep = [177,177,177,177,177,177,177,177,177,177]

qtcs_nondom_gzip = [41, 40, 39, 40, 41, 41, 40, 41, 39, 41]
div_ga_nondom_gzip = [105, 105, 105, 105, 105, 105, 105, 105, 105, 105]
qaoa_nondom_gzip = [100, 100, 76, 99, 101, 76, 99, 99, 101, 99]
add_greedy_nondom_gzip = [26,26,26,26,26,26,26,26,26,26]

qtcs_nondom_sed = [86, 86, 86, 86, 86, 86, 86, 86, 86, 86]
div_ga_nondom_sed = [89, 62, 103, 101, 89, 97, 97, 94, 101, 103]
qaoa_nondom_sed = [183, 183, 183, 183, 183, 183, 183, 183, 183, 183]
add_greedy_nondom_sed = [80,80,80,80,80,80,80,80,80,80]

NOISE 2%
qtcs_nondom_flex = [187, 187, 187, 187, 187, 187, 187, 187, 187, 187]
div_ga_nondom_flex = [140, 140, 140, 140, 140, 140, 140, 140, 140, 140]
qaoa_nondom_flex = [492, 491, 492, 491, 492, 492, 491, 492, 491, 492]
add_greedy_nondom_flex = [205,205,205,205,205,205,205,205,205,205]

qtcs_nondom_grep = [192, 192, 193, 192, 193, 192, 193, 192, 193, 193]
div_ga_nondom_grep = [70, 70, 70, 70, 70, 70, 70, 70, 70, 70]
qaoa_nondom_grep = [321, 321, 321, 321, 321, 321, 321, 321, 321, 321]
add_greedy_nondom_grep = [177,177,177,177,177,177,177,177,177,177]

qtcs_nondom_gzip = [41, 40, 39, 40, 41, 41, 40, 41, 39, 41]
div_ga_nondom_gzip = [105, 105, 105, 105, 105, 105, 105, 105, 105, 105]
qaoa_nondom_gzip = [99, 75, 77, 76, 99, 99, 99, 76, 76, 99]
add_greedy_nondom_gzip = [26,26,26,26,26,26,26,26,26,26]

qtcs_nondom_sed = [86, 86, 86, 86, 86, 86, 86, 86, 86, 86]
div_ga_nondom_sed = [89, 62, 103, 101, 89, 97, 97, 94, 101, 103]
qaoa_nondom_sed = [183, 183, 183, 183, 183, 183, 183, 183, 183, 183]
add_greedy_nondom_sed = [80,80,80,80,80,80,80,80,80,80]

NOISE 5%
qtcs_nondom_flex = [187, 187, 187, 187, 187, 187, 187, 187, 187, 187]
div_ga_nondom_flex = [140, 140, 140, 140, 140, 140, 140, 140, 140, 140]
qaoa_nondom_flex = [492, 491, 492, 491, 492, 492, 491, 492, 491, 492]
add_greedy_nondom_flex = [205,205,205,205,205,205,205,205,205,205]

qtcs_nondom_grep = [192, 192, 193, 192, 193, 192, 193, 192, 193, 193]
div_ga_nondom_grep = [70, 70, 70, 70, 70, 70, 70, 70, 70, 70]
qaoa_nondom_grep = [321, 321, 321, 321, 321, 321, 321, 321, 321, 321]
add_greedy_nondom_grep = [177,177,177,177,177,177,177,177,177,177]

qtcs_nondom_gzip = [41, 40, 39, 40, 41, 41, 40, 41, 39, 41]
div_ga_nondom_gzip = [105, 105, 105, 105, 105, 105, 105, 105, 105, 105]
qaoa_nondom_gzip = [78, 76, 100, 78, 77, 77, 77, 98, 76, 100]
add_greedy_nondom_gzip = [26,26,26,26,26,26,26,26,26,26]

qtcs_nondom_sed = [86, 86, 86, 86, 86, 86, 86, 86, 86, 86]
div_ga_nondom_sed = [89, 62, 103, 101, 89, 97, 97, 94, 101, 103]
qaoa_nondom_sed = [183, 183, 183, 183, 183, 183, 183, 183, 183, 183]
add_greedy_nondom_sed = [80,80,80,80,80,80,80,80,80,80]
"""

In [ ]:
#the static values below have been manually stored after each run of the experiments

qtcs_nondom_flex = [187, 187, 187, 187, 187, 187, 187, 187, 187, 187]
div_ga_nondom_flex = [140, 140, 140, 140, 140, 140, 140, 140, 140, 140]
qaoa_nondom_flex = [492, 491, 492, 491, 492, 492, 491, 492, 491, 492]
add_greedy_nondom_flex = [205,205,205,205,205,205,205,205,205,205]

qtcs_nondom_grep = [192, 192, 193, 192, 193, 192, 193, 192, 193, 193]
div_ga_nondom_grep = [70, 70, 70, 70, 70, 70, 70, 70, 70, 70]
qaoa_nondom_grep = [321, 321, 321, 321, 321, 321, 321, 321, 321, 321]
add_greedy_nondom_grep = [177,177,177,177,177,177,177,177,177,177]

qtcs_nondom_gzip = [41, 40, 39, 40, 41, 41, 40, 41, 39, 41]
div_ga_nondom_gzip = [105, 105, 105, 105, 105, 105, 105, 105, 105, 105]
qaoa_nondom_gzip = [78, 76, 100, 78, 77, 77, 77, 98, 76, 100]
add_greedy_nondom_gzip = [26,26,26,26,26,26,26,26,26,26]

qtcs_nondom_sed = [86, 86, 86, 86, 86, 86, 86, 86, 86, 86]
div_ga_nondom_sed = [89, 62, 103, 101, 89, 97, 97, 94, 101, 103]
qaoa_nondom_sed = [183, 183, 183, 183, 183, 183, 183, 183, 183, 183]
add_greedy_nondom_sed = [80,80,80,80,80,80,80,80,80,80]

lists = [qtcs_nondom_flex,div_ga_nondom_flex,qaoa_nondom_flex,qtcs_nondom_grep,div_ga_nondom_grep,qaoa_nondom_grep,qtcs_nondom_gzip,div_ga_nondom_gzip,qaoa_nondom_gzip,qtcs_nondom_sed,div_ga_nondom_sed,qaoa_nondom_sed]

# we are interested in knowing if the obtained sequences are normally distributed to decide what statistical test we should apply
for i, list in enumerate(lists, start=1):
    stat, p_value = shapiro(list)
    print(f"List {i}: Statistic = {stat:.5f}, P-value = {p_value:.5f}")
    
    # check if the sequence is normally distributed (using 0.05 as threshold)
    if p_value > 0.05:
        print(f"List {i} seems to be normally distributed (p-value = {p_value:.5f})")
    else:
        print(f"List {i} NOT seems to be normally distributed (p-value = {p_value:.5f})")

In [ ]:
def a12(lst1,lst2,rev=True):
  "how often is x in lst1 more than y in lst2?"
  more = same = 0.0
  for x in lst1:
    for y in lst2:
      if x==y : same += 1
      elif rev and x > y : more += 1
      elif not rev and x < y : more += 1
  return (more + 0.5*same)  / (len(lst1)*len(lst2))

def stat_test(app1, app2):
    statistic, pvalue = mannwhitneyu(app1, app2, alternative='greater')

    # Calculate the A12 effect size using Vargha and Delaney's formula
    a12_effect_size = a12(app1, app2)

    return pvalue, a12_effect_size

p_value, a_12 = stat_test(div_ga_nondom_gzip,qaoa_nondom_gzip)
print("p_value: " + str(p_value))
print("a12: " + str(a_12))
#5.8493119157503726e-05

In [ ]:
with open('./results/add-greedy/sed_data.json', 'r') as file:
    pareto_fronts_json = json.load(file)

greedy_execution_time = pareto_fronts_json['resolution_time(ms)']
print("Greedy Execution Time")
print(greedy_execution_time)

with open('results/selectqa/old/sed-data.json', 'r') as file:
    pareto_fronts_json = json.load(file)

qtcs_qpu_execution_time = pareto_fronts_json['mean_qpu_run_time(ms)']
qtcs_pareto_building_time = pareto_fronts_json['mean_pareto_fronts_building_time(ms)']
print("SelectQA QPU Execution Time")
print(qtcs_qpu_execution_time)
print("SelectQA Pareto Building Time")
print(qtcs_pareto_building_time)

with open('./results/divga/sed_pareto_fronts_divga.json', 'r') as file:
    pareto_fronts_json = json.load(file)

divga_execution_time = pareto_fronts_json['DIVGA_II_mean_execution_time_ms']
print("DIV-GA Execution Time")
print(divga_execution_time)

"""with open('results/selectqaoa/statevector_sim/sed-data.json', 'r') as file:
    pareto_fronts_json = json.load(file)

qaoa_qpu_execution_time = pareto_fronts_json['mean_qpu_run_time(ms)']
qaoa_pareto_building_time = pareto_fronts_json['mean_pareto_fronts_building_time(ms)']
print(qaoa_qpu_execution_time)
print(qaoa_pareto_building_time)"""

In [ ]:
with open('results/selectqaoa/ideal/sed-data.json', 'r') as file:
    pareto_fronts_json = json.load(file)

qaoa_qpu_execution_time = pareto_fronts_json['mean_qpu_run_time(ms)']
qaoa_pareto_building_time = pareto_fronts_json['mean_pareto_fronts_building_time(ms)']
print("SelectQAOA Execution Time (Ideal)")
print(qaoa_qpu_execution_time)
print("SelectQAOA Pareto Building Time (Ideal)")
print(qaoa_pareto_building_time)

with open('results/selectqaoa/fake_vigo/sed-data.json', 'r') as file:
    pareto_fronts_json = json.load(file)

qaoa_qpu_execution_time = pareto_fronts_json['mean_qpu_run_time(ms)']
qaoa_pareto_building_time = pareto_fronts_json['mean_pareto_fronts_building_time(ms)']
print("SelectQAOA Execution Time (Fake Vigo)")
print(qaoa_qpu_execution_time)
print("SelectQAOA Pareto Building Time (Fake Vigo)")
print(qaoa_pareto_building_time)

with open('results/selectqaoa/depolarizing_sim/01/sed-data.json', 'r') as file:
    pareto_fronts_json = json.load(file)

qaoa_qpu_execution_time = pareto_fronts_json['mean_qpu_run_time(ms)']
qaoa_pareto_building_time = pareto_fronts_json['mean_pareto_fronts_building_time(ms)']
print("SelectQAOA Execution Time (Depolarizing Error 1%)")
print(qaoa_qpu_execution_time)
print("SelectQAOA Pareto Building Time (Depolarizing Error 1%)")
print(qaoa_pareto_building_time)

with open('results/selectqaoa/depolarizing_sim/02/sed-data.json', 'r') as file:
    pareto_fronts_json = json.load(file)

qaoa_qpu_execution_time = pareto_fronts_json['mean_qpu_run_time(ms)']
qaoa_pareto_building_time = pareto_fronts_json['mean_pareto_fronts_building_time(ms)']
print("SelectQAOA Execution Time (Depolarizing Error 2%)")
print(qaoa_qpu_execution_time)
print("SelectQAOA Pareto Building Time (Depolarizing Error 2%)")
print(qaoa_pareto_building_time)

with open('results/selectqaoa/depolarizing_sim/05/sed-data.json', 'r') as file:
    pareto_fronts_json = json.load(file)

qaoa_qpu_execution_time = pareto_fronts_json['mean_qpu_run_time(ms)']
qaoa_pareto_building_time = pareto_fronts_json['mean_pareto_fronts_building_time(ms)']
print("SelectQAOA Execution Time (Depolarizing Error 5%)")
print(qaoa_qpu_execution_time)
print("SelectQAOA Pareto Building Time (Depolarizing Error 5%)")
print(qaoa_pareto_building_time)

## BootQA

BootQA is the first approach to a test case optimization problem using quantum computing. BootQA relies on real-world industrial datasets:

- PaintControl: dataset gathered from ABB Robotics Norway;
- GSDTSR: dataset gathered from Google.

 Each test case in both the datasets has values related to the properties:
 
- "execution time": time needed by the test case to be executed;
- "failure rate": the ability, in percentage, of a test case to spot a failure.

To deal with physical limitations of quantum annealers, BootQA randomly divides the problem into m sub-problems of size n; once it has obtained the m sub-solutions, BootQA merges them into one final solution.

## Evaluation Metrics

To compare SelectQA and BootQA were executed 10 indipendent experiments for each dataset. The resulting suites (10 for each dataset) were directly confronted in terms of "execution time" and "failure rate". The statistical tests used the same methods used for the previous comparisons.

To compare the efficiency of the two methods, has been use the "QPU Access Time" metric.

In [ ]:
def bootstrap_confidence_interval(data, num_samples, confidence_alpha=0.95):
    """This function determines the statistical range within we would expect the mean value of execution times to fall; it relies on the bootstrapping strategy, which allows the calculation of the confidence interval by repeated sampling (with replacement) from the existing data to obtain an estimate of the confidence interval."""
    sample_means = []
    for _ in range(num_samples):
        bootstrap_sample = [random.choice(data) for _ in range(len(data))]
        sample_mean = np.mean(bootstrap_sample)
        sample_means.append(sample_mean)
    
    lower_percentile = (1 - confidence_alpha) / 2 * 100
    upper_percentile = (confidence_alpha + (1 - confidence_alpha) / 2) * 100
    lower_bound = np.percentile(sample_means, lower_percentile)
    upper_bound = np.percentile(sample_means, upper_percentile)
    
    return lower_bound, upper_bound

In [ ]:
def make_linear_terms_bootqa(cluster_test_cases, test_cases_costs, test_cases_rates, alpha):
    max_cost = max(test_cases_costs)
    
    estimated_costs = []

    #linear coefficients, that are the diagonal of the matrix encoding the QUBO
    for test_case in cluster_test_cases:
        estimated_costs.append((alpha * ((test_cases_costs[test_case])/max_cost)) - ((1-alpha)*test_cases_rates[test_case]))
    
    return np.array(estimated_costs)

In [ ]:
def create_linear_qubo(linear_terms):
    """This function is the one that has to encode the QUBO problem that QAOA will have to solve. The QUBO problem specifies the optimization to solve and a quadratic binary unconstrained problem"""
    qubo = QuadraticProgram()
    
    for i in range(0,len(linear_terms)):
        qubo.binary_var('x%s' % (i))

    qubo.minimize(linear=linear_terms)

    return qubo

In [ ]:
def get_data(data_name):
    data = pd.read_csv("datasets/boot_qa_datasets/" + data_name + ".csv", dtype={"time": float, "rate": float})
    data = data[data['rate'] > 0]
    return data

bootqa_programs = ["gsdtsr","paintcontrol"]
bootqa_clusters = dict()

for bootqa_program in bootqa_programs:
    data = get_data(bootqa_program)
    
    # Total suite metrics
    test_cases_costs = data["time"].tolist()
    test_cases_rates = data["rate"].tolist()
    print(f"Tot suite cost: {sum(test_cases_costs)}")
    print(f"Tot suite rate: {sum(test_cases_rates)}")
        
    # Normalize data
    cluster_data = np.column_stack((test_cases_costs, test_cases_rates))
    scaler = StandardScaler()
    normalized_data = scaler.fit_transform(cluster_data)
    
    if bootqa_program == "gsdtsr":
        num_clusters = 40
    else:
        num_clusters = 20
    
    start = time.time()
    linkage_matrix = linkage(normalized_data, method='ward')
    clusters = fcluster(linkage_matrix, t=num_clusters, criterion='maxclust')
    
    # Organize test cases by cluster and split large clusters
    clustered_data = defaultdict(list)
    for idx, cluster_id in enumerate(clusters):
        clustered_data[cluster_id].append(idx)
        
    # Process clusters to split large ones into 3 parts if they exceed 30 elements
    new_cluster_id = max(clustered_data.keys()) + 1  # Start new IDs after existing ones
    to_add = []  # To collect new smaller clusters for adding
    
    for cluster_id, elements in list(clustered_data.items()):  # Use list() to avoid modifying during iteration
        if len(elements) > 24:
            # Split into 3 parts
            split_size = max(1, len(elements) // 3)  # Calculate split size, ensuring minimum size of 1
            parts = [elements[i:i + split_size] for i in range(0, len(elements), split_size)]
    
            # Ensure we end up with exactly 3 parts
            while len(parts) < 3:
                parts.append([])  # Add empty parts if fewer than 3
    
            # Add new parts to to_add list
            to_add.extend(parts[:3])
    
            # Remove original large cluster
            del clustered_data[cluster_id]
    
    # Add new clusters to clustered_data with new IDs
    for part in to_add:
        if part:  # Only add if the part is non-empty
            clustered_data[new_cluster_id] = part
            new_cluster_id += 1
    end = time.time()
    print("SelectQAOA Decomposition Time(ms): " + str((end-start)*1000))
    
    bootqa_clusters[bootqa_program] = clustered_data
    
    # Step 3: Calculate the metrics for each refined cluster
    cluster_metrics = {}
    for cluster_id, members in clustered_data.items():
        tot_cluster_costs = sum(test_cases_costs[i] for i in members)
        tot_cluster_rates = sum(test_cases_rates[i] for i in members)
        cluster_metrics[cluster_id] = {
            "tot_cluster_cost": tot_cluster_costs,
            "tot_cluster_rates": tot_cluster_rates
        }
        print(f"Cluster {cluster_id + 1} metrics:")
        print(f"Test Cases: {members}")
        print(f" - Num. Test Cases: {len(members):.2f}")
        print(f" - Execution Cost: {tot_cluster_costs:.2f}")
        print(f" - Failure Rate: {tot_cluster_rates}")
    
    print("===========================================================================")    
    
    for cluster_id in clustered_data.keys():
        if len(clustered_data[cluster_id]) > 24:
            print("Program: " + bootqa_program)
            print("Test cases of cluster " + str(cluster_id) + ": " + str(len(clustered_data[cluster_id])))
    
    # Plotting the clusters in 3D space
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    
    # Extracting data for plotting
    exec_costs = np.array(test_cases_costs)
    rates = np.array(test_cases_rates)
    
    # Plot each refined cluster with a different color
    colors = plt.cm.get_cmap("tab10", len(clustered_data))  # A colormap with as many colors as clusters
    for cluster_id, members in clustered_data.items():
        ax.scatter(
            exec_costs[members], 
            rates[members], 
            color=colors(cluster_id % 10), 
            label=f"Cluster {cluster_id + 1}"
        )
    
    # Label the axes
    ax.set_xlabel("Execution Cost")
    ax.set_ylabel("Failure Rate")
    ax.legend()
    ax.set_title("Test Case Clustering Visualization")
    
    # Display the plot
    plt.show()


### Statevector Simulator

In [ ]:
bootqa_alphas = {"gsdtsr": 98,"paintcontrol": 0.90}
run_times_dictionary = {"gsdtsr": [],"paintcontrol": []}

algorithm_globals.random_seed = 10598
backend = Aer.get_backend("statevector_simulator")
qaoa_mes = QAOA(sampler=BackendSampler(backend=backend), optimizer=COBYLA(100), reps=1)
qaoa = MinimumEigenOptimizer(qaoa_mes)

for bootqa_program in bootqa_programs:
    data = get_data(bootqa_program)
    # Total suite metrics
    test_cases_costs = data["time"].tolist()
    test_cases_rates = data["rate"].tolist()
    
    final_test_suite_costs = []
    final_failure_rates = []
    for i in range(10):
        final_selected_cases = []
        cluster_number = 0
        for cluster_id in bootqa_clusters[bootqa_program]:
            print("Cluster: " + str(bootqa_clusters[bootqa_program][cluster_id]))
            linear_terms = make_linear_terms_bootqa(bootqa_clusters[bootqa_program][cluster_id], test_cases_costs, test_cases_rates, bootqa_alphas[bootqa_program])
            linear_qubo = create_linear_qubo(linear_terms)
            print("Linear QUBO: " + str(linear_qubo))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(linear_qubo)
            print("QAOA Result: " + str(qaoa_result))
            e = time.time()
            run_times_dictionary[bootqa_program].append((e-s)*1000)
            
            variable_values = qaoa_result.x
            indexes_selected_cases = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_cases))
            selected_tests = []
            for index in indexes_selected_cases:
                selected_tests.append(bootqa_clusters[bootqa_program][cluster_id][index])
            print("Selected tests: " + str(selected_tests))
            for selected_test in selected_tests:
                if selected_test not in final_test_suite_costs:
                    final_selected_cases.append(selected_test)
            
        #compute the final test suite cost
        final_test_suite_cost = 0
        for selected_test_case in final_selected_cases:
            final_test_suite_cost += test_cases_costs[selected_test_case]
        final_test_suite_costs.append(final_test_suite_cost)
            
        #compute the total failure rate
        final_failure_rate = 0
        for selected_test_case in final_selected_cases:
            final_failure_rate += test_cases_rates[selected_test_case]
        final_failure_rates.append(final_failure_rate)
    
    print("Final Test Suite: " + str(final_selected_cases))
    #compute the qpu access times
    qpu_run_times_without_zeros = []
    for access_time in run_times_dictionary[bootqa_program]:
      if access_time != 0:
        qpu_run_times_without_zeros.append(access_time)
    lower_bound, upper_bound = bootstrap_confidence_interval(qpu_run_times_without_zeros, 1000, 0.95)
    for i in range(len(run_times_dictionary[bootqa_program])):
      if run_times_dictionary[bootqa_program][i] == 0:
          run_times_dictionary[bootqa_program][i] = upper_bound
    average_qpu_access_time = statistics.mean(run_times_dictionary[bootqa_program]) 
    
    var_names = ["final_test_suite_costs", "final_failure_rates",
                 "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)", "all_qpu_access_times(ms)",
                 "qpu_lower_bound(ms)", "qpu_upper_bound()"]
    values = [final_test_suite_costs, final_failure_rates, average_qpu_access_time, 
              statistics.stdev(run_times_dictionary[bootqa_program]), run_times_dictionary[bootqa_program],
              lower_bound, upper_bound]
    
    # Ensure the directory exists
    output_dir = "results/selectqaoa/statevector_sim"
    os.makedirs(output_dir, exist_ok=True)
    
    # Path to save the file
    file_path = os.path.join(output_dir, f"{bootqa_program}.csv")
    
    # Writing results to the file
    with open(file_path, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(var_names)
        writer.writerow(values)
    print(f"Results saved to {file_path}")

## Ideal Aer Simulator

In [ ]:
bootqa_alphas = {"gsdtsr": 0.98,"paintcontrol": 0.90}
run_times_dictionary = {"gsdtsr": [],"paintcontrol": []}

sim_ideal = AerSimulator()

algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=BackendSampler(backend=sim_ideal), optimizer=COBYLA(100), reps=1)
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA

for bootqa_program in bootqa_programs:
    data = get_data(bootqa_program)
    # Total suite metrics
    test_cases_costs = data["time"].tolist()
    test_cases_rates = data["rate"].tolist()
    
    final_test_suite_costs = []
    final_failure_rates = []
    for i in range(10):
        final_selected_cases = []
        cluster_number = 0
        for cluster_id in bootqa_clusters[bootqa_program]:
            print("Cluster: " + str(bootqa_clusters[bootqa_program][cluster_id]))
            linear_terms = make_linear_terms_bootqa(bootqa_clusters[bootqa_program][cluster_id], test_cases_costs, test_cases_rates, bootqa_alphas[bootqa_program])
            linear_qubo = create_linear_qubo(linear_terms)
            print("Linear QUBO: " + str(linear_qubo))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(linear_qubo)
            print("QAOA Result: " + str(qaoa_result))
            e = time.time()
            run_times_dictionary[bootqa_program].append((e-s)*1000)
            
            variable_values = qaoa_result.x
            indexes_selected_cases = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_cases))
            selected_tests = []
            for index in indexes_selected_cases:
                selected_tests.append(bootqa_clusters[bootqa_program][cluster_id][index])
            print("Selected tests: " + str(selected_tests))
            for selected_test in selected_tests:
                if selected_test not in final_test_suite_costs:
                    final_selected_cases.append(selected_test)
            
        #compute the final test suite cost
        final_test_suite_cost = 0
        for selected_test_case in final_selected_cases:
            final_test_suite_cost += test_cases_costs[selected_test_case]
        final_test_suite_costs.append(final_test_suite_cost)
            
        #compute the total failure rate
        final_failure_rate = 0
        for selected_test_case in final_selected_cases:
            final_failure_rate += test_cases_rates[selected_test_case]
        final_failure_rates.append(final_failure_rate)
    
    print("Final Test Suite: " + str(final_selected_cases))
    #compute the qpu access times
    qpu_run_times_without_zeros = []
    for access_time in run_times_dictionary[bootqa_program]:
      if access_time != 0:
        qpu_run_times_without_zeros.append(access_time)
    lower_bound, upper_bound = bootstrap_confidence_interval(qpu_run_times_without_zeros, 1000, 0.95)
    for i in range(len(run_times_dictionary[bootqa_program])):
      if run_times_dictionary[bootqa_program][i] == 0:
          run_times_dictionary[bootqa_program][i] = upper_bound
    average_qpu_access_time = statistics.mean(run_times_dictionary[bootqa_program]) 
    
    var_names = ["final_test_suite_costs", "final_failure_rates",
                 "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)", "all_qpu_access_times(ms)",
                 "qpu_lower_bound(ms)", "qpu_upper_bound()"]
    values = [final_test_suite_costs, final_failure_rates, average_qpu_access_time, 
              statistics.stdev(run_times_dictionary[bootqa_program]), run_times_dictionary[bootqa_program],
              lower_bound, upper_bound]
    
    # Ensure the directory exists
    output_dir = "results/selectqaoa/ideal"
    os.makedirs(output_dir, exist_ok=True)
    
    # Path to save the file
    file_path = os.path.join(output_dir, f"{bootqa_program}.csv")
    
    # Writing results to the file
    with open(file_path, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(var_names)
        writer.writerow(values)
    print(f"Results saved to {file_path}")

## Fake Vigo Noise Simulator

In [ ]:
bootqa_alphas = {"gsdtsr": 0.98,"paintcontrol": 0.90}
run_times_dictionary = {"gsdtsr": [],"paintcontrol": []}

device_backend = FakeVigoV2()

algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=BackendSampler(backend=device_backend), optimizer=COBYLA(100), reps=1)
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
bootqa_programs = ["gsdtsr"]
for bootqa_program in bootqa_programs:
    data = get_data(bootqa_program)
    # Total suite metrics
    test_cases_costs = data["time"].tolist()
    test_cases_rates = data["rate"].tolist()
    
    final_test_suite_costs = []
    final_failure_rates = []
    for i in range(10):
        final_selected_cases = []
        cluster_number = 0
        for cluster_id in bootqa_clusters[bootqa_program]:
            print("Cluster: " + str(bootqa_clusters[bootqa_program][cluster_id]))
            linear_terms = make_linear_terms_bootqa(bootqa_clusters[bootqa_program][cluster_id], test_cases_costs, test_cases_rates, bootqa_alphas[bootqa_program])
            linear_qubo = create_linear_qubo(linear_terms)
            print("Linear QUBO: " + str(linear_qubo))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(linear_qubo)
            print("Program: " + str(bootqa_program) + ". It: " + str(i))
            print("QAOA Result: " + str(qaoa_result))
            e = time.time()
            run_times_dictionary[bootqa_program].append((e-s)*1000)
            
            variable_values = qaoa_result.x
            indexes_selected_cases = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_cases))
            selected_tests = []
            for index in indexes_selected_cases:
                selected_tests.append(bootqa_clusters[bootqa_program][cluster_id][index])
            print("Selected tests: " + str(selected_tests))
            for selected_test in selected_tests:
                if selected_test not in final_test_suite_costs:
                    final_selected_cases.append(selected_test)
            
        #compute the final test suite cost
        final_test_suite_cost = 0
        for selected_test_case in final_selected_cases:
            final_test_suite_cost += test_cases_costs[selected_test_case]
        final_test_suite_costs.append(final_test_suite_cost)
            
        #compute the total failure rate
        final_failure_rate = 0
        for selected_test_case in final_selected_cases:
            final_failure_rate += test_cases_rates[selected_test_case]
        final_failure_rates.append(final_failure_rate)
    
    print("Final Test Suite: " + str(final_selected_cases))
    #compute the qpu access times
    qpu_run_times_without_zeros = []
    for access_time in run_times_dictionary[bootqa_program]:
      if access_time != 0:
        qpu_run_times_without_zeros.append(access_time)
    lower_bound, upper_bound = bootstrap_confidence_interval(qpu_run_times_without_zeros, 1000, 0.95)
    for i in range(len(run_times_dictionary[bootqa_program])):
      if run_times_dictionary[bootqa_program][i] == 0:
          run_times_dictionary[bootqa_program][i] = upper_bound
    average_qpu_access_time = statistics.mean(run_times_dictionary[bootqa_program]) 
    
    var_names = ["final_test_suite_costs", "final_failure_rates",
                 "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)", "all_qpu_access_times(ms)",
                 "qpu_lower_bound(ms)", "qpu_upper_bound()"]
    values = [final_test_suite_costs, final_failure_rates, average_qpu_access_time, 
              statistics.stdev(run_times_dictionary[bootqa_program]), run_times_dictionary[bootqa_program],
              lower_bound, upper_bound]
    
    # Ensure the directory exists
    output_dir = "results/selectqaoa/fake_vigo"
    os.makedirs(output_dir, exist_ok=True)
    
    # Path to save the file
    file_path = os.path.join(output_dir, f"{bootqa_program}.csv")
    
    # Writing results to the file
    with open(file_path, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(var_names)
        writer.writerow(values)
    print(f"Results saved to {file_path}")

## Depolarizing Error Simulator

In [ ]:
bootqa_alphas = {"gsdtsr": 0.98,"paintcontrol": 0.90}
run_times_dictionary = {"gsdtsr": [],"paintcontrol": []}

noise_dep = NoiseModel()

# Define a 1% depolarizing error for single-qubit and two-qubit gates
depolarizing_error_1q = depolarizing_error(0.01, 1)  # 1% depolarizing error for single-qubit gates
depolarizing_error_2q = depolarizing_error(0.01, 2)  # 1% depolarizing error for two-qubit gates

# Add the 1% depolarizing noise to the single-qubit gates (rx and rz)
noise_dep.add_all_qubit_quantum_error(depolarizing_error_1q, ['rx', 'rz'])

# Add the 1% depolarizing noise to the two-qubit gate (cx)
noise_dep.add_all_qubit_quantum_error(depolarizing_error_2q, ['cx'])

sim_dep = AerSimulator(noise_model=noise_dep)

algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=BackendSampler(backend=sim_dep), optimizer=COBYLA(100), reps=1)
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
bootqa_programs = ["gsdtsr"]
for bootqa_program in bootqa_programs:
    data = get_data(bootqa_program)
    # Total suite metrics
    test_cases_costs = data["time"].tolist()
    test_cases_rates = data["rate"].tolist()
    
    final_test_suite_costs = []
    final_failure_rates = []
    for i in range(10):
        final_selected_cases = []
        cluster_number = 0
        for cluster_id in bootqa_clusters[bootqa_program]:
            print("Cluster: " + str(bootqa_clusters[bootqa_program][cluster_id]))
            linear_terms = make_linear_terms_bootqa(bootqa_clusters[bootqa_program][cluster_id], test_cases_costs, test_cases_rates, bootqa_alphas[bootqa_program])
            linear_qubo = create_linear_qubo(linear_terms)
            print("Linear QUBO: " + str(linear_qubo))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(linear_qubo)
            print("Program: " + str(bootqa_program) + ". It: " + str(i))
            print("QAOA Result: " + str(qaoa_result))
            e = time.time()
            run_times_dictionary[bootqa_program].append((e-s)*1000)
            
            variable_values = qaoa_result.x
            indexes_selected_cases = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_cases))
            selected_tests = []
            for index in indexes_selected_cases:
                selected_tests.append(bootqa_clusters[bootqa_program][cluster_id][index])
            print("Selected tests: " + str(selected_tests))
            for selected_test in selected_tests:
                if selected_test not in final_test_suite_costs:
                    final_selected_cases.append(selected_test)
            
        #compute the final test suite cost
        final_test_suite_cost = 0
        for selected_test_case in final_selected_cases:
            final_test_suite_cost += test_cases_costs[selected_test_case]
        final_test_suite_costs.append(final_test_suite_cost)
            
        #compute the total failure rate
        final_failure_rate = 0
        for selected_test_case in final_selected_cases:
            final_failure_rate += test_cases_rates[selected_test_case]
        final_failure_rates.append(final_failure_rate)
    
    print("Final Test Suite: " + str(final_selected_cases))
    #compute the qpu access times
    qpu_run_times_without_zeros = []
    for access_time in run_times_dictionary[bootqa_program]:
      if access_time != 0:
        qpu_run_times_without_zeros.append(access_time)
    lower_bound, upper_bound = bootstrap_confidence_interval(qpu_run_times_without_zeros, 1000, 0.95)
    for i in range(len(run_times_dictionary[bootqa_program])):
      if run_times_dictionary[bootqa_program][i] == 0:
          run_times_dictionary[bootqa_program][i] = upper_bound
    average_qpu_access_time = statistics.mean(run_times_dictionary[bootqa_program]) 
    
    var_names = ["final_test_suite_costs", "final_failure_rates",
                 "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)", "all_qpu_access_times(ms)",
                 "qpu_lower_bound(ms)", "qpu_upper_bound()"]
    values = [final_test_suite_costs, final_failure_rates, average_qpu_access_time, 
              statistics.stdev(run_times_dictionary[bootqa_program]), run_times_dictionary[bootqa_program],
              lower_bound, upper_bound]
    
    # Ensure the directory exists
    output_dir = "results/selectqaoa/depolarizing_sim/01"
    os.makedirs(output_dir, exist_ok=True)
    
    # Path to save the file
    file_path = os.path.join(output_dir, f"{bootqa_program}.csv")
    
    # Writing results to the file
    with open(file_path, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(var_names)
        writer.writerow(values)
    print(f"Results saved to {file_path}")

In [ ]:
bootqa_alphas = {"gsdtsr": 0.98,"paintcontrol": 0.90}
run_times_dictionary = {"gsdtsr": [],"paintcontrol": []}

noise_dep = NoiseModel()

# Define a 1% depolarizing error for single-qubit and two-qubit gates
depolarizing_error_1q = depolarizing_error(0.02, 1)  # 2% depolarizing error for single-qubit gates
depolarizing_error_2q = depolarizing_error(0.02, 2)  # 2% depolarizing error for two-qubit gates

# Add the 1% depolarizing noise to the single-qubit gates (rx and rz)
noise_dep.add_all_qubit_quantum_error(depolarizing_error_1q, ['rx', 'rz'])

# Add the 1% depolarizing noise to the two-qubit gate (cx)
noise_dep.add_all_qubit_quantum_error(depolarizing_error_2q, ['cx'])

sim_dep = AerSimulator(noise_model=noise_dep)

algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=BackendSampler(backend=sim_dep), optimizer=COBYLA(100), reps=1)
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
bootqa_programs = ["gsdtsr"]
for bootqa_program in bootqa_programs:
    data = get_data(bootqa_program)
    # Total suite metrics
    test_cases_costs = data["time"].tolist()
    test_cases_rates = data["rate"].tolist()
    
    final_test_suite_costs = []
    final_failure_rates = []
    for i in range(10):
        final_selected_cases = []
        cluster_number = 0
        for cluster_id in bootqa_clusters[bootqa_program]:
            print("Cluster: " + str(bootqa_clusters[bootqa_program][cluster_id]))
            linear_terms = make_linear_terms_bootqa(bootqa_clusters[bootqa_program][cluster_id], test_cases_costs, test_cases_rates, bootqa_alphas[bootqa_program])
            linear_qubo = create_linear_qubo(linear_terms)
            print("Linear QUBO: " + str(linear_qubo))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(linear_qubo)
            print("Program: " + str(bootqa_program) + ". It: " + str(i))
            print("QAOA Result: " + str(qaoa_result))
            e = time.time()
            run_times_dictionary[bootqa_program].append((e-s)*1000)
            
            variable_values = qaoa_result.x
            indexes_selected_cases = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_cases))
            selected_tests = []
            for index in indexes_selected_cases:
                selected_tests.append(bootqa_clusters[bootqa_program][cluster_id][index])
            print("Selected tests: " + str(selected_tests))
            for selected_test in selected_tests:
                if selected_test not in final_test_suite_costs:
                    final_selected_cases.append(selected_test)
            
        #compute the final test suite cost
        final_test_suite_cost = 0
        for selected_test_case in final_selected_cases:
            final_test_suite_cost += test_cases_costs[selected_test_case]
        final_test_suite_costs.append(final_test_suite_cost)
            
        #compute the total failure rate
        final_failure_rate = 0
        for selected_test_case in final_selected_cases:
            final_failure_rate += test_cases_rates[selected_test_case]
        final_failure_rates.append(final_failure_rate)
    
    print("Final Test Suite: " + str(final_selected_cases))
    #compute the qpu access times
    qpu_run_times_without_zeros = []
    for access_time in run_times_dictionary[bootqa_program]:
      if access_time != 0:
        qpu_run_times_without_zeros.append(access_time)
    lower_bound, upper_bound = bootstrap_confidence_interval(qpu_run_times_without_zeros, 1000, 0.95)
    for i in range(len(run_times_dictionary[bootqa_program])):
      if run_times_dictionary[bootqa_program][i] == 0:
          run_times_dictionary[bootqa_program][i] = upper_bound
    average_qpu_access_time = statistics.mean(run_times_dictionary[bootqa_program]) 
    
    var_names = ["final_test_suite_costs", "final_failure_rates",
                 "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)", "all_qpu_access_times(ms)",
                 "qpu_lower_bound(ms)", "qpu_upper_bound()"]
    values = [final_test_suite_costs, final_failure_rates, average_qpu_access_time, 
              statistics.stdev(run_times_dictionary[bootqa_program]), run_times_dictionary[bootqa_program],
              lower_bound, upper_bound]
    
    # Ensure the directory exists
    output_dir = "results/selectqaoa/depolarizing_sim/02"
    os.makedirs(output_dir, exist_ok=True)
    
    # Path to save the file
    file_path = os.path.join(output_dir, f"{bootqa_program}.csv")
    
    # Writing results to the file
    with open(file_path, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(var_names)
        writer.writerow(values)
    print(f"Results saved to {file_path}")

In [ ]:
bootqa_alphas = {"gsdtsr": 0.98,"paintcontrol": 0.90}
run_times_dictionary = {"gsdtsr": [],"paintcontrol": []}

noise_dep = NoiseModel()

# Define a 1% depolarizing error for single-qubit and two-qubit gates
depolarizing_error_1q = depolarizing_error(0.05, 1)  # 5% depolarizing error for single-qubit gates
depolarizing_error_2q = depolarizing_error(0.05, 2)  # 5% depolarizing error for two-qubit gates

# Add the 1% depolarizing noise to the single-qubit gates (rx and rz)
noise_dep.add_all_qubit_quantum_error(depolarizing_error_1q, ['rx', 'rz'])

# Add the 1% depolarizing noise to the two-qubit gate (cx)
noise_dep.add_all_qubit_quantum_error(depolarizing_error_2q, ['cx'])

sim_dep = AerSimulator(noise_model=noise_dep)

algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=BackendSampler(backend=sim_dep), optimizer=COBYLA(100), reps=1)
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
bootqa_programs = ["gsdtsr"]
for bootqa_program in bootqa_programs:
    data = get_data(bootqa_program)
    # Total suite metrics
    test_cases_costs = data["time"].tolist()
    test_cases_rates = data["rate"].tolist()
    
    final_test_suite_costs = []
    final_failure_rates = []
    for i in range(10):
        final_selected_cases = []
        cluster_number = 0
        for cluster_id in bootqa_clusters[bootqa_program]:
            print("Cluster: " + str(bootqa_clusters[bootqa_program][cluster_id]))
            linear_terms = make_linear_terms_bootqa(bootqa_clusters[bootqa_program][cluster_id], test_cases_costs, test_cases_rates, bootqa_alphas[bootqa_program])
            linear_qubo = create_linear_qubo(linear_terms)
            print("Linear QUBO: " + str(linear_qubo))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(linear_qubo)
            print("Program: " + str(bootqa_program) + ". It: " + str(i))
            print("QAOA Result: " + str(qaoa_result))
            e = time.time()
            run_times_dictionary[bootqa_program].append((e-s)*1000)
            
            variable_values = qaoa_result.x
            indexes_selected_cases = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_cases))
            selected_tests = []
            for index in indexes_selected_cases:
                selected_tests.append(bootqa_clusters[bootqa_program][cluster_id][index])
            print("Selected tests: " + str(selected_tests))
            for selected_test in selected_tests:
                if selected_test not in final_test_suite_costs:
                    final_selected_cases.append(selected_test)
            
        #compute the final test suite cost
        final_test_suite_cost = 0
        for selected_test_case in final_selected_cases:
            final_test_suite_cost += test_cases_costs[selected_test_case]
        final_test_suite_costs.append(final_test_suite_cost)
            
        #compute the total failure rate
        final_failure_rate = 0
        for selected_test_case in final_selected_cases:
            final_failure_rate += test_cases_rates[selected_test_case]
        final_failure_rates.append(final_failure_rate)
    
    print("Final Test Suite: " + str(final_selected_cases))
    #compute the qpu access times
    qpu_run_times_without_zeros = []
    for access_time in run_times_dictionary[bootqa_program]:
      if access_time != 0:
        qpu_run_times_without_zeros.append(access_time)
    lower_bound, upper_bound = bootstrap_confidence_interval(qpu_run_times_without_zeros, 1000, 0.95)
    for i in range(len(run_times_dictionary[bootqa_program])):
      if run_times_dictionary[bootqa_program][i] == 0:
          run_times_dictionary[bootqa_program][i] = upper_bound
    average_qpu_access_time = statistics.mean(run_times_dictionary[bootqa_program]) 
    
    var_names = ["final_test_suite_costs", "final_failure_rates",
                 "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)", "all_qpu_access_times(ms)",
                 "qpu_lower_bound(ms)", "qpu_upper_bound()"]
    values = [final_test_suite_costs, final_failure_rates, average_qpu_access_time, 
              statistics.stdev(run_times_dictionary[bootqa_program]), run_times_dictionary[bootqa_program],
              lower_bound, upper_bound]
    
    # Ensure the directory exists
    output_dir = "results/selectqaoa/depolarizing_sim/05"
    os.makedirs(output_dir, exist_ok=True)
    
    # Path to save the file
    file_path = os.path.join(output_dir, f"{bootqa_program}.csv")
    
    # Writing results to the file
    with open(file_path, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(var_names)
        writer.writerow(values)
    print(f"Results saved to {file_path}")